In [1]:
from datetime import datetime
import os 
import pandas as pd
import numpy as np

In [2]:
bucket = os.getenv("WORKSPACE_BUCKET")

USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')

In [3]:
%env USER_NAME={USER_NAME}

env: USER_NAME=williamsjacr


In [4]:
!pip3 install --upgrade dsub


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
%%writefile ~/aou_dsub.bash
#!/bin/bash
function aou_dsub () {

  # Get a shorter username to leave more characters for the job name.
  local DSUB_USER_NAME="$(echo "${OWNER_EMAIL}" | cut -d@ -f1)"

  # For AoU RWB projects network name is "network".
  local AOU_NETWORK=network
  local AOU_SUBNETWORK=subnetwork

  dsub \
      --provider google-cls-v2 \
      --user-project "${GOOGLE_PROJECT}"\
      --project "${GOOGLE_PROJECT}"\
      --image 'marketplace.gcr.io/google/ubuntu1804:latest' \
      --network "${AOU_NETWORK}" \
      --subnetwork "${AOU_SUBNETWORK}" \
      --service-account "$(gcloud config get-value account)" \
      --user "${DSUB_USER_NAME}" \
      --regions us-central1 \
      --logging "${WORKSPACE_BUCKET}/dsub/logs/{job-name}/{user-id}/$(date +'%Y%m%d/%H%M%S')/{job-id}-{task-id}-{task-attempt}.log" \
      "$@"
}

Writing /home/jupyter/aou_dsub.bash


In [6]:
!echo source ~/aou_dsub.bash >> ~/.bashrc

In [93]:
%%writefile NullModel.R
rm(list = ls())

all_phenotype_File <- commandArgs(TRUE)[1]
print(all_phenotype_File)

gds_file <- commandArgs(TRUE)[2]
print(gds_file)

OUTPUT_PATH <- commandArgs(TRUE)[3]
print(OUTPUT_PATH)

library(gdsfmt)
library(SeqArray)
library(SeqVarTools)
library(dplyr)
library(STAAR)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(Matrix)
library(SCANG)
library(STAARpipeline)

trait <- "TC"

agds.path <- gds_file

genofile <- seqOpen(agds.path)

id.genotype <- seqGetData(genofile,"sample.id")

seqClose(genofile)

print(str(id.genotype))

all_phenotypes <- read.csv(all_phenotype_File)

colnames(all_phenotypes)[10] <- "sex"

print(str(all_phenotypes))

all_phenotypes <- all_phenotypes[all_phenotypes$IID %in% id.genotype,]

print(nrow(all_phenotypes))

all_phenotypes <- all_phenotypes[!is.na(all_phenotypes[,trait]),]
  
print(nrow(all_phenotypes))

all_phenotypes$age2 <- all_phenotypes$age^2

all_phenotypes$TCadj_resid <- resid(lm(TC~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10, data = all_phenotypes))

all_phenotypes$TCadj_norm <- sd(all_phenotypes$TC)*scale(qnorm((rank(all_phenotypes$TCadj_resid,na.last="keep")-0.5)/length(all_phenotypes$TCadj_resid)))

obj.STAAR.UKB <- fit_nullmodel(TCadj_norm~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10, data = all_phenotypes,id = "IID",kins = NULL,family = gaussian(link = "identity"))
  
save(obj.STAAR.UKB,file = paste0(OUTPUT_PATH,"/TC_Null_Model.RData"))


Overwriting NullModel.R


In [94]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

tasks <- rbind(tasks, data.frame(
  '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/NullModel.R",
  '--input all_phenotype_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/all_phenotypes.csv",
  '--input gds_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataAGDS/exome_v7.1/exome.chr21.gds",
  '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/",
  check.names = FALSE
))

write.table(tasks,
            file="score_task.txt",
            row.names=F, col.names=T,
            sep='\t', quote=F)

Overwriting score_task.R


In [95]:
%%writefile NullModel.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${all_phenotype_File} ${gds_file} ${OUTPUT_PATH}

Overwriting NullModel.sh


In [96]:
!Rscript score_task.R

In [30]:
!gsutil -m cp NullModel.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/

Copying file://NullModel.R [Content-Type=application/octet-stream]...
/ [1/1 files][  1.4 KiB/  1.4 KiB] 100% Done                                    
Operation completed over 1 objects/1.4 KiB.                                      


In [31]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 100 \
  --boot-disk-size 25 \
  --min-ram 16 \
  --timeout "48h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script NullModel.sh \
  --tasks score_task.txt

Job properties:
  job-id: nullmodel--williamsjacr--240910-152518-10
  job-name: nullmodel
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/9054957783243937729
Launched job-id: nullmodel--williamsjacr--240910-152518-10
1 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'nullmodel--williamsjacr--240910-152518-10' --users 'williamsjacr' --status '*'
To cancel the job, run:
  ddel --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'nullmodel--williamsjacr--240910-152518-10' --users 'williamsjacr'


In [33]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'nullmodel--williamsjacr--240910-152518-10' --users 'williamsjacr' --status '*'

Job Name      Task  Status    Last Update
----------  ------  --------  --------------
nullmodel        1  Success   09-10 15:30:18



In [97]:
%%writefile Meta_Coding_AoU.R
rm(list=ls())
gc()

arrayid <- as.numeric(commandArgs(TRUE)[1])

Null_Model <- commandArgs(TRUE)[2]
print(Null_Model)

Annotation_name_catalog <- commandArgs(TRUE)[3]
print(Annotation_name_catalog)

GDS_File <- commandArgs(TRUE)[4]
print(GDS_File)

OUTPUT_PATH <- commandArgs(TRUE)[5]
print(OUTPUT_PATH)

time <- system.time({
  ## load required packages
  library(gdsfmt)
  library(SeqArray)
  library(SeqVarTools)
  library(STAAR)
  library(STAARpipeline)
  library(readr)
  library(dplyr)
  library(stringr)
  
  library(devtools)
#  build(R_PACKAGE,path = "")
#  install.packages("MetaSTAAR_0.9.6.3.tar.gz", repos = NULL, type="source")
  library(MetaSTAAR)
  library(expm)
  library(Matrix)
  
  MetaSTAARlite_worker_sumstat <- function(genotype,obj_nullmodel,variant_info,qc_label=NULL,
                                           annotation_phred=NULL,for_individual_analysis=FALSE){
    
    if(!inherits(genotype, "matrix") && !inherits(genotype, "Matrix")){
      stop("genotype is not a matrix!")
    }
    
    if(inherits(genotype, "sparseMatrix")){
      genotype <- as.matrix(genotype)
    }
    
    if(dim(genotype)[2] != dim(variant_info)[1]){
      stop(paste0("Dimensions don't match for genotype and variant_info!"))
    }
    
    if(!is.null(qc_label) && dim(variant_info)[1] != length(qc_label)){
      stop(paste0("Dimensions don't match for variant_info and qc_label!"))
    }
    
    if(!is.null(annotation_phred) && dim(annotation_phred)[1] != dim(variant_info)[1]){
      stop(paste0("Dimensions don't match for annotation_phred and variant_info!"))
    }
    
    N <- dim(genotype)[1]
    alt_AC <- as.integer(colSums(2 - genotype))
    MAC <- as.integer(pmin(alt_AC, 2 * N - alt_AC))
    genotype <- matrix_flip(genotype)
    MAF <- genotype$MAF
    variant_label <- as.vector(MAF>0)
    Geno <- genotype$Geno[,variant_label,drop=FALSE]
    Geno <- as(Geno,"dgCMatrix")
    rm(genotype)
    gc()
    
    if(obj_nullmodel$relatedness){
      if(!obj_nullmodel$sparse_kins){
        stop(paste0("Please use a sparse genetic relatedness matrix when fitting the null model!"))
      }
    }else{
      if(obj_nullmodel$family[1] == "binomial"){
        obj_nullmodel$Sigma_i <- Diagonal(x = obj_nullmodel$weights)
      }else if(obj_nullmodel$family[1] == "gaussian"){
        obj_nullmodel$Sigma_i <- Diagonal(length(obj_nullmodel$y)) / summary(obj_nullmodel)$dispersion
      }
      obj_nullmodel$Sigma_iX <- obj_nullmodel$Sigma_i %*% model.matrix(obj_nullmodel)
      obj_nullmodel$cov <- solve(t(model.matrix(obj_nullmodel)) %*% obj_nullmodel$Sigma_i %*% model.matrix(obj_nullmodel))
      obj_nullmodel$scaled.residuals <- (obj_nullmodel$y - obj_nullmodel$fitted.values) / summary(obj_nullmodel)$dispersion
    }
    
    GTSinvX_cov <- as(t(Geno) %*% obj_nullmodel$Sigma_iX,"matrix") %*% sqrtm(obj_nullmodel$cov)
    #V <- diag(t(Geno) %*% obj_nullmodel$Sigma_i %*% Geno - GTSinvX_cov %*% t(GTSinvX_cov))
    V <- colSums(Geno * (obj_nullmodel$Sigma_i %*% Geno)) - rowSums(GTSinvX_cov^2) # faster for large number of variants
    U <- as.vector(t(Geno) %*% obj_nullmodel$scaled.residuals)
    rm(Geno)
    gc()
    
    if (!is.null(qc_label)){
      sumstat <- data.frame(variant_info[,c("chr","pos","ref","alt")],qc_label,alt_AC,MAC,MAF,N)
    }else{
      sumstat <- data.frame(variant_info[,c("chr","pos","ref","alt")],alt_AC,MAC,MAF,N)
    }
    if (!is.null(annotation_phred)){
      sumstat <- cbind(sumstat,annotation_phred)
    }
    sumstat <- sumstat[variant_label,]
    if(!for_individual_analysis){
      sumstat <- cbind(sumstat,U,V,GTSinvX_cov)
    }else{
      sumstat <- cbind(sumstat,U,V)
    }
    
    return(sumstat)
  }
  
  MetaSTAARlite_worker_cov <- function(genotype,obj_nullmodel,cov_maf_cutoff=0.05,
                                       qc_label=NULL,signif.digits=3){
    
    if(!inherits(genotype, "matrix") && !inherits(genotype, "Matrix")){
      stop("genotype is not a matrix!")
    }
    
    if(cov_maf_cutoff < 0 | cov_maf_cutoff > 0.5){
      stop("cov_maf_cutoff should be a number between 0 and 0.5!")
    }
    
    if(cov_maf_cutoff == 0.5){
      cov_maf_cutoff <- 0.5 + 1e-16
    }
    
    if(inherits(genotype, "sparseMatrix")){
      MAF <- colMeans(genotype)/2
      if (!is.null(qc_label)){
        RV_label <- as.vector((MAF<cov_maf_cutoff)&(MAF>0)&(qc_label=="PASS"))
      }else{
        RV_label <- as.vector((MAF<cov_maf_cutoff)&(MAF>0))
      }
      Geno_rare <- genotype[,RV_label,drop=FALSE]
      Geno_rare <- as(Geno_rare,"dgCMatrix")
    }else{
      genotype <- matrix_flip(genotype)
      MAF <- genotype$MAF
      if (!is.null(qc_label)){
        RV_label <- as.vector((MAF<cov_maf_cutoff)&(MAF>0)&(qc_label=="PASS"))
      }else{
        RV_label <- as.vector((MAF<cov_maf_cutoff)&(MAF>0))
      }
      Geno_rare <- genotype$Geno[,RV_label,drop=FALSE]
      Geno_rare <- as(Geno_rare,"dgCMatrix")
    }
    
    rm(genotype,MAF)
    gc()
    
    if(obj_nullmodel$relatedness){
      if(!obj_nullmodel$sparse_kins){
        stop(paste0("Please use a sparse genetic relatedness matrix when fitting the null model!"))
      }
    }else{
      if(obj_nullmodel$family[1] == "binomial"){
        obj_nullmodel$Sigma_i <- Diagonal(x = obj_nullmodel$weights)
      }else if(obj_nullmodel$family[1] == "gaussian"){
        obj_nullmodel$Sigma_i <- Diagonal(length(obj_nullmodel$y)) / summary(obj_nullmodel)$dispersion
      }
      obj_nullmodel$Sigma_iX <- obj_nullmodel$Sigma_i %*% model.matrix(obj_nullmodel)
      obj_nullmodel$cov <- solve(t(model.matrix(obj_nullmodel)) %*% obj_nullmodel$Sigma_i %*% model.matrix(obj_nullmodel))
      obj_nullmodel$scaled.residuals <- (obj_nullmodel$y - obj_nullmodel$fitted.values) / summary(obj_nullmodel)$dispersion
    }
    
    GTSinvG_rare <- t(obj_nullmodel$Sigma_i %*% Geno_rare) %*% Geno_rare
    rm(Geno_rare)
    gc()
    
    GTSinvG_rare <- as(GTSinvG_rare,"TsparseMatrix")
    remove_ind <- (GTSinvG_rare@j < GTSinvG_rare@i) # Be careful about multi-allelic issue
    GTSinvG_rare@i <- GTSinvG_rare@i[!remove_ind]
    GTSinvG_rare@j <- GTSinvG_rare@j[!remove_ind]
    GTSinvG_rare@x <- GTSinvG_rare@x[!remove_ind]
    rm(remove_ind)
    gc()
    GTSinvG_rare <- as(GTSinvG_rare,"CsparseMatrix")
    
    ### Create a version of GTSinvG_rare with rounded significant digits
    if(!is.null(signif.digits)){
      GTSinvG_rare <- signif(GTSinvG_rare, digits = signif.digits)
    }
    
    return(GTSinvG_rare)
  }
  
  
  coding_MetaSTAARlite_worker <- function(chr,gene_name,genofile,obj_nullmodel,genes,known_loci=NULL,
                                          cov_maf_cutoff=0.05,signif.digits=NULL,
                                          QC_label="annotation/filter",check_qc_label=FALSE,variant_type=c("SNV","Indel","variant"),
                                          Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                                          Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                                          silent=FALSE){
    
    ## evaluate choices
    variant_type <- match.arg(variant_type)
    
    phenotype.id <- obj_nullmodel$id_include
    
    filter <- seqGetData(genofile, QC_label)
    if(variant_type=="variant")
    {
      if(check_qc_label)
      {
        SNVlist <- TRUE
      }else
      {
        SNVlist <- filter == "PASS"
      }
    }
    
    if(variant_type=="SNV")
    {
      if(check_qc_label)
      {
        SNVlist <- isSNV(genofile)
      }else
      {
        SNVlist <- (filter == "PASS") & isSNV(genofile)
      }
    }
    
    if(variant_type=="Indel")
    {
      if(check_qc_label)
      {
        SNVlist <- !isSNV(genofile)
      }else
      {
        SNVlist <- (filter == "PASS") & (!isSNV(genofile))
      }
    }
    
    position <- as.numeric(seqGetData(genofile, "position"))
    variant.id <- seqGetData(genofile, "variant.id")
    
    if(!is.null(known_loci))
    {
      allele <- seqGetData(genofile, "allele")
      
      loc_SNV <- c()
      for (i in 1:dim(known_loci)[1]){
        loc_SNV <- c(loc_SNV,which((position==known_loci$POS[i])&(allele==paste0(known_loci$REF[i],",",known_loci$ALT[i]))))
      }
      
      seqSetFilter(genofile,variant.id=variant.id[loc_SNV],sample.id=phenotype.id)
      
      G_SNV <- seqGetData(genofile, "$dosage")
      
      if (!is.null(G_SNV)){
        id.SNV <- seqGetData(genofile,"sample.id")
        id.SNV.match <- rep(0,length(id.SNV))
        
        for(i in 1:length(id.SNV))
        {
          id.SNV.match[i] <- which.max(id.SNV==phenotype.id[i])
        }
        
        G_SNV <- G_SNV[id.SNV.match,,drop=FALSE]
        G_SNV_MAF <- apply(G_SNV, 2, function(x){sum(x[!is.na(x)])/2/sum(!is.na(x))})
        for (i in 1:length(G_SNV_MAF)){
          if (G_SNV_MAF[i] <= 0.5){
            G_SNV[is.na(G_SNV[,i]),i] <- 0
          }
          else{
            G_SNV[is.na(G_SNV[,i]),i] <- 2
          }
        }
        
        pos_adj <- as.integer(seqGetData(genofile, "position"))
        ref_adj <- as.character(seqGetData(genofile, "$ref"))
        alt_adj <- as.character(seqGetData(genofile, "$alt"))
        variant_adj_info <- data.frame(chr,pos_adj,ref_adj,alt_adj)
        colnames(variant_adj_info) <- c("chr","pos","ref","alt")
        variant_adj_info
        
        seqResetFilter(genofile)
      }
    }
    
    rm(filter)
    gc()
    
    ### Gene
    kk <- which(genes[,1]==gene_name)
    
    sub_start_loc <- genes[kk,3]
    sub_end_loc <- genes[kk,4]
    
    is.in <- (SNVlist)&(position>=sub_start_loc)&(position<=sub_end_loc)
    variant.id.gene <- variant.id[is.in]
    
    rm(position)
    gc()
    
    seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
    
    ## Gencode_Exonic
    GENCODE.EXONIC.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.EXONIC.Category")]))
    ## Gencode
    GENCODE.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Category")]))
    ## Meta.SVM.Pred
    MetaSVM_pred <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="MetaSVM")]))
    
    ################################################
    #           Coding
    ################################################
    variant.id.gene <- seqGetData(genofile, "variant.id")
    lof.in.coding <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|(GENCODE.Category=="ncRNA_splicing")|(GENCODE.Category=="ncRNA_exonic;splicing")|(GENCODE.EXONIC.Category=="nonsynonymous SNV")|(GENCODE.EXONIC.Category=="synonymous SNV")
    variant.id.gene <- variant.id.gene[lof.in.coding]
    
    seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
    
    ## Gencode_Exonic
    GENCODE.EXONIC.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.EXONIC.Category")]))
    ## Gencode
    GENCODE.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Category")]))
    ## Meta.SVM.Pred
    MetaSVM_pred <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="MetaSVM")]))
    
    ## Annotation
    Anno.Int.PHRED.sub <- NULL
    Anno.Int.PHRED.sub.name <- NULL
    
    if(variant_type=="SNV")
    {
      if(Use_annotation_weights)
      {
        for(k in 1:length(Annotation_name))
        {
          if(Annotation_name[k]%in%Annotation_name_catalog$name)
          {
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
            Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
            
            if(Annotation_name[k]=="CADD")
            {
              Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
            }
            Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
          }
        }
        
        Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
        colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
      }
    }
    
    ################################################
    #                  plof_ds
    ################################################
    variant.id.gene <- seqGetData(genofile, "variant.id")
    lof.in.plof <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|(GENCODE.Category=="ncRNA_splicing")|(GENCODE.Category=="ncRNA_exonic;splicing")|((GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D"))
    variant.id.gene.category <- variant.id.gene[lof.in.plof]
    
    seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
    
    pos <- as.integer(seqGetData(genofile, "position"))
    ref <- as.character(seqGetData(genofile, "$ref"))
    alt <- as.character(seqGetData(genofile, "$alt"))
    if(check_qc_label){
      qc_label <- as.character(seqGetData(genofile, QC_label))
    }else{
      qc_label <- NULL
    }
    
    ## genotype id
    id.genotype <- seqGetData(genofile,"sample.id")
    # id.genotype.match <- rep(0,length(id.genotype))
    
    id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
    phenotype.id.merge <- data.frame(phenotype.id)
    phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
    id.genotype.match <- phenotype.id.merge$index
    
    ## Genotype
    Geno <- seqGetData(genofile, "$dosage")
    Geno <- Geno[id.genotype.match,,drop=FALSE]
    
    summary_stat_list <- list()
    GTSinvG_rare_list <- list()
    cov_cond_list <- list()
    
    summary_stat <- NULL
    GTSinvG_rare <- NULL
    cov_cond <- NULL
    
    if(!is.null(Geno))
    {
      # Summary statistics
      genotype <- matrix_impute(Geno)
      variant_info <- data.frame(chr,pos,ref,alt,row.names=NULL)
      
      ## Annotation
      Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.plof,]
      
      try(summary_stat <- MetaSTAARlite_worker_sumstat(genotype,obj_nullmodel,variant_info,qc_label,
                                                       Anno.Int.PHRED.sub.category),silent=silent)
      
      # Covariance matrices
      genotype <- matrix_flip(Geno)$Geno
      genotype <- as(genotype,"dgCMatrix")
      
      rm(Geno)
      gc()
      
      try(GTSinvG_rare <- MetaSTAARlite_worker_cov(genotype,obj_nullmodel,cov_maf_cutoff,
                                                   qc_label,signif.digits),silent=silent)
      
      # Covariance matrices for conditional analysis
      if(!is.null(known_loci))
      {
        try(cov_cond <- MetaSTAARlite_worker_cov_cond(genotype,G_SNV,obj_nullmodel,variant_info,variant_adj_info),silent=silent)
      }
    }
    
    summary_stat_list[["plof_ds"]] <- summary_stat
    GTSinvG_rare_list[["plof_ds"]] <- GTSinvG_rare
    cov_cond_list[["plof_ds"]] <- cov_cond
    
    #####################################################
    #                      plof
    #####################################################
    lof.in.plof <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|(GENCODE.Category=="ncRNA_splicing")|(GENCODE.Category=="ncRNA_exonic;splicing")
    variant.id.gene.category <- variant.id.gene[lof.in.plof]
    
    seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
    
    pos <- as.integer(seqGetData(genofile, "position"))
    ref <- as.character(seqGetData(genofile, "$ref"))
    alt <- as.character(seqGetData(genofile, "$alt"))
    if(check_qc_label){
      qc_label <- as.character(seqGetData(genofile, QC_label))
    }else{
      qc_label <- NULL
    }
    
    ## genotype id
    id.genotype <- seqGetData(genofile,"sample.id")
    # id.genotype.match <- rep(0,length(id.genotype))
    
    id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
    phenotype.id.merge <- data.frame(phenotype.id)
    phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
    id.genotype.match <- phenotype.id.merge$index
    
    ## Genotype
    Geno <- seqGetData(genofile, "$dosage")
    Geno <- Geno[id.genotype.match,,drop=FALSE]
    
    summary_stat <- NULL
    GTSinvG_rare <- NULL
    cov_cond <- NULL
    
    if(!is.null(Geno))
    {
      # Summary statistics
      genotype <- matrix_impute(Geno)
      variant_info <- data.frame(chr,pos,ref,alt,row.names=NULL)
      
      ## Annotation
      Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.plof,]
      
      try(summary_stat <- MetaSTAARlite_worker_sumstat(genotype,obj_nullmodel,variant_info,qc_label,
                                                       Anno.Int.PHRED.sub.category),silent=silent)
      
      # Covariance matrices
      genotype <- matrix_flip(Geno)$Geno
      genotype <- as(genotype,"dgCMatrix")
      
      rm(Geno)
      gc()
      
      try(GTSinvG_rare <- MetaSTAARlite_worker_cov(genotype,obj_nullmodel,cov_maf_cutoff,
                                                   qc_label,signif.digits),silent=silent)
      
      # Covariance matrices for conditional analysis
      if(!is.null(known_loci))
      {
        try(cov_cond <- MetaSTAARlite_worker_cov_cond(genotype,G_SNV,obj_nullmodel,variant_info,variant_adj_info),silent=silent)
      }
    }
    
    summary_stat_list[["plof"]] <- summary_stat
    GTSinvG_rare_list[["plof"]] <- GTSinvG_rare
    cov_cond_list[["plof"]] <- cov_cond
    
    #############################################
    #             synonymous
    #############################################
    lof.in.synonymous <- (GENCODE.EXONIC.Category=="synonymous SNV")
    variant.id.gene.category <- variant.id.gene[lof.in.synonymous]
    
    seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
    
    pos <- as.integer(seqGetData(genofile, "position"))
    ref <- as.character(seqGetData(genofile, "$ref"))
    alt <- as.character(seqGetData(genofile, "$alt"))
    if(check_qc_label){
      qc_label <- as.character(seqGetData(genofile, QC_label))
    }else{
      qc_label <- NULL
    }
    
    ## genotype id
    id.genotype <- seqGetData(genofile,"sample.id")
    # id.genotype.match <- rep(0,length(id.genotype))
    
    id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
    phenotype.id.merge <- data.frame(phenotype.id)
    phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
    id.genotype.match <- phenotype.id.merge$index
    
    ## Genotype
    Geno <- seqGetData(genofile, "$dosage")
    Geno <- Geno[id.genotype.match,,drop=FALSE]
    
    summary_stat <- NULL
    GTSinvG_rare <- NULL
    cov_cond <- NULL
    
    if(!is.null(Geno))
    {
      # Summary statistics
      genotype <- matrix_impute(Geno)
      variant_info <- data.frame(chr,pos,ref,alt,row.names=NULL)
      
      ## Annotation
      Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.synonymous,]
      
      try(summary_stat <- MetaSTAARlite_worker_sumstat(genotype,obj_nullmodel,variant_info,qc_label,
                                                       Anno.Int.PHRED.sub.category),silent=silent)
      
      # Covariance matrices
      genotype <- matrix_flip(Geno)$Geno
      genotype <- as(genotype,"dgCMatrix")
      
      rm(Geno)
      gc()
      
      try(GTSinvG_rare <- MetaSTAARlite_worker_cov(genotype,obj_nullmodel,cov_maf_cutoff,
                                                   qc_label,signif.digits),silent=silent)
      
      # Covariance matrices for conditional analysis
      if(!is.null(known_loci))
      {
        try(cov_cond <- MetaSTAARlite_worker_cov_cond(genotype,G_SNV,obj_nullmodel,variant_info,variant_adj_info),silent=silent)
      }
    }
    
    summary_stat_list[["synonymous"]] <- summary_stat
    GTSinvG_rare_list[["synonymous"]] <- GTSinvG_rare
    cov_cond_list[["synonymous"]] <- cov_cond
    
    #################################################
    #        missense
    #################################################
    lof.in.missense <- (GENCODE.EXONIC.Category=="nonsynonymous SNV")
    variant.id.gene.category <- variant.id.gene[lof.in.missense]
    
    seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
    
    pos <- as.integer(seqGetData(genofile, "position"))
    ref <- as.character(seqGetData(genofile, "$ref"))
    alt <- as.character(seqGetData(genofile, "$alt"))
    if(check_qc_label){
      qc_label <- as.character(seqGetData(genofile, QC_label))
    }else{
      qc_label <- NULL
    }
    
    ## genotype id
    id.genotype <- seqGetData(genofile,"sample.id")
    # id.genotype.match <- rep(0,length(id.genotype))
    
    id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
    phenotype.id.merge <- data.frame(phenotype.id)
    phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
    id.genotype.match <- phenotype.id.merge$index
    
    ## Genotype
    Geno <- seqGetData(genofile, "$dosage")
    Geno <- Geno[id.genotype.match,,drop=FALSE]
    
    summary_stat <- NULL
    GTSinvG_rare <- NULL
    cov_cond <- NULL
    
    if(!is.null(Geno))
    {
      # Summary statistics
      genotype <- matrix_impute(Geno)
      variant_info <- data.frame(chr,pos,ref,alt,row.names=NULL)
      
      ## Annotation
      Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.missense,]
      
      try(summary_stat <- MetaSTAARlite_worker_sumstat(genotype,obj_nullmodel,variant_info,qc_label,
                                                       Anno.Int.PHRED.sub.category),silent=silent)
      
      # Covariance matrices
      genotype <- matrix_flip(Geno)$Geno
      genotype <- as(genotype,"dgCMatrix")
      
      rm(Geno)
      gc()
      
      try(GTSinvG_rare <- MetaSTAARlite_worker_cov(genotype,obj_nullmodel,cov_maf_cutoff,
                                                   qc_label,signif.digits),silent=silent)
      
      # Covariance matrices for conditional analysis
      if(!is.null(known_loci))
      {
        try(cov_cond <- MetaSTAARlite_worker_cov_cond(genotype,G_SNV,obj_nullmodel,variant_info,variant_adj_info),silent=silent)
      }
    }
    
    summary_stat_list[["missense"]] <- summary_stat
    GTSinvG_rare_list[["missense"]] <- GTSinvG_rare
    cov_cond_list[["missense"]] <- cov_cond
    
    #################################################
    #         disruptive missense
    #################################################
    lof.in.dmissense <- (GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D")
    variant.id.gene.category <- variant.id.gene[lof.in.dmissense]
    
    seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
    
    pos <- as.integer(seqGetData(genofile, "position"))
    ref <- as.character(seqGetData(genofile, "$ref"))
    alt <- as.character(seqGetData(genofile, "$alt"))
    if(check_qc_label){
      qc_label <- as.character(seqGetData(genofile, QC_label))
    }else{
      qc_label <- NULL
    }
    
    ## genotype id
    id.genotype <- seqGetData(genofile,"sample.id")
    # id.genotype.match <- rep(0,length(id.genotype))
    
    id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
    phenotype.id.merge <- data.frame(phenotype.id)
    phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
    id.genotype.match <- phenotype.id.merge$index
    
    ## Genotype
    Geno <- seqGetData(genofile, "$dosage")
    Geno <- Geno[id.genotype.match,,drop=FALSE]
    
    summary_stat <- NULL
    GTSinvG_rare <- NULL
    cov_cond <- NULL
    
    if(!is.null(Geno))
    {
      # Summary statistics
      genotype <- matrix_impute(Geno)
      variant_info <- data.frame(chr,pos,ref,alt,row.names=NULL)
      
      ## Annotation
      Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.dmissense,]
      
      try(summary_stat <- MetaSTAARlite_worker_sumstat(genotype,obj_nullmodel,variant_info,qc_label,
                                                       Anno.Int.PHRED.sub.category),silent=silent)
      
      # Covariance matrices
      genotype <- matrix_flip(Geno)$Geno
      genotype <- as(genotype,"dgCMatrix")
      
      rm(Geno)
      gc()
      
      try(GTSinvG_rare <- MetaSTAARlite_worker_cov(genotype,obj_nullmodel,cov_maf_cutoff,
                                                   qc_label,signif.digits),silent=silent)
      
      # Covariance matrices for conditional analysis
      if(!is.null(known_loci))
      {
        try(cov_cond <- MetaSTAARlite_worker_cov_cond(genotype,G_SNV,obj_nullmodel,variant_info,variant_adj_info),silent=silent)
      }
    }
    
    summary_stat_list[["disruptive_missense"]] <- summary_stat
    GTSinvG_rare_list[["disruptive_missense"]] <- GTSinvG_rare
    cov_cond_list[["disruptive_missense"]] <- cov_cond
    
    seqResetFilter(genofile)
    
    if(!is.null(known_loci))
    {
      return(list(summary_stat_list=summary_stat_list,
                  GTSinvG_rare_list=GTSinvG_rare_list,
                  cov_cond_list=cov_cond_list))
    }else
    {
      return(list(summary_stat_list=summary_stat_list,
                  GTSinvG_rare_list=GTSinvG_rare_list))
    }
  }
  
  
  ###########################################################
  #           User Input
  ###########################################################
  ## Null model
  obj_nullmodel <- get(load(Null_Model))

## Parameter
QC_label <- "annotation/filter"
geno_missing_imputation <- "mean"
variant_type <- "SNV"

## Annotation_dir
Annotation_dir <- "annotation/info/FunctionalAnnotation"
## Annotation channel
Annotation_name_catalog <- read.csv(Annotation_name_catalog)
## Use_annotation_weights
Use_annotation_weights <- TRUE
## Annotation name
Annotation_name <- c("CADD","LINSIGHT","FATHMM.XF","aPC.EpigeneticActive","aPC.EpigeneticRepressed","aPC.EpigeneticTranscription",
                     "aPC.Conservation","aPC.LocalDiversity","aPC.Mappability","aPC.TF","aPC.Protein")

  ## output file name
  output_file_name <- paste0(OUTPUT_PATH,"/AoU_TC_Coding")
  
  ###########################################################
  #           Main Function 
  ###########################################################
  ## gene number in job
  gene_num_in_array <- 50
  group.num.allchr <- ceiling(table(genes_info[,2])/gene_num_in_array)
  sum(group.num.allchr)
  
  chr <- which.max(arrayid <= cumsum(group.num.allchr))
  group.num <- group.num.allchr[chr]
  
  if (chr == 1){
    groupid <- arrayid
  }else{
    groupid <- arrayid - cumsum(group.num.allchr)[chr-1]
  }
  
  genes_info_chr <- genes_info[genes_info[,2]==chr,]
  sub_seq_num <- dim(genes_info_chr)[1]
  
  if(groupid < group.num){
    sub_seq_id <- ((groupid - 1)*gene_num_in_array + 1):(groupid*gene_num_in_array)
  }else{
    sub_seq_id <- ((groupid - 1)*gene_num_in_array + 1):sub_seq_num
  }
  
  ## exclude large coding masks
  if(arrayid==57){
    sub_seq_id <- setdiff(sub_seq_id,840)
  }
  
  if(arrayid==112){
    sub_seq_id <- setdiff(sub_seq_id,c(543,544))
  }
  
  if(arrayid==113){
    sub_seq_id <- setdiff(sub_seq_id,c(575,576,577,578,579,580,582))
  }
  
  ## aGDS file
  agds.path <- GDS_File

  genofile <- seqOpen(agds.path)
  
  genes <- genes_info
  
  coding_sumstat <- list()
  coding_cov <- list()
  for(kk in sub_seq_id){
    print(kk)
    gene_name <- genes_info_chr[kk,1]
    results_temp <- coding_MetaSTAARlite_worker(chr=chr,gene_name=gene_name,genofile=genofile,obj_nullmodel=obj_nullmodel,genes=genes,
                                                cov_maf_cutoff=0.05,signif.digits=NULL,
                                                QC_label=QC_label,check_qc_label=TRUE,variant_type=variant_type,
                                                Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                                                Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name)
    coding_sumstat[[gene_name]] <- results_temp$summary_stat_list
    coding_cov[[gene_name]] <- results_temp$GTSinvG_rare_list
  }
  
  save(coding_sumstat,file=paste0(output_file_name,"_sumstat_",arrayid,".Rdata"),compress = "xz")
  save(coding_cov,file=paste0(output_file_name,"_cov_",arrayid,".Rdata"),compress = "xz")
  
  seqClose(genofile)
  
})[3]

save(time,file = paste0(output_file_name,"_time_",arrayid,".Rdata"))

Overwriting Meta_Coding_AoU.R


In [98]:
%%writefile Meta_Coding_AoU.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${arrayid} ${Null_Model} ${Annotation_name_catalog} ${GDS_File} ${OUTPUT_PATH}

Overwriting Meta_Coding_AoU.sh


In [99]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

for(arrayid in c(112,113,332)){
        if(arrayid < 41){
            chr <- 1
        }else if(arrayid < 66){
            chr <- 2
        }else if(arrayid < 87){
            chr <- 3
        }else if(arrayid < 102){
            chr <- 4
        }else if(arrayid < 120){
            chr <- 5
        }else if(arrayid < 141){
            chr <- 6
        }else if(arrayid < 159){
            chr <- 7
        }else if(arrayid < 173){
            chr <- 8
        }else if(arrayid < 189){
            chr <- 9
        }else if(arrayid < 204){
            chr <- 10
        }else if(arrayid < 230){
            chr <- 11
        }else if(arrayid < 250){
            chr <- 12
        }else if(arrayid < 257){
            chr <- 13
        }else if(arrayid < 269){
            chr <- 14
        }else if(arrayid < 281){
            chr <- 15
        }else if(arrayid < 298){
            chr <- 16
        }else if(arrayid < 321){
            chr <- 17
        }else if(arrayid < 327){
            chr <- 18
        }else if(arrayid < 355){
            chr <- 19
        }else if(arrayid < 366){
            chr <- 20
        }else if(arrayid < 371){
            chr <- 21
        }else{
            chr <- 22
        }
        
        tasks <- rbind(tasks, data.frame(
            '--env arrayid'=arrayid,
            '--input Null_Model'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/TC_Null_Model.RData",
            '--input Annotation_name_catalog'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataGDS/acaf_threshold_v7/Annotation_name_catalog.csv",
            '--input GDS_File'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataAGDS/exome_v7.1/exome.chr",chr,".gds"),
            '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/Meta_Coding_AoU.R",
            '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/AoU_Sumstats/",
            check.names = FALSE
        )) 
    }
  
write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [100]:
!Rscript score_task.R

In [101]:
!gsutil -m cp Meta_Coding_AoU.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/

Copying file://Meta_Coding_AoU.R [Content-Type=application/octet-stream]...
/ [1/1 files][ 28.4 KiB/ 28.4 KiB] 100% Done                                    
Operation completed over 1 objects/28.4 KiB.                                     


In [102]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 100 \
  --boot-disk-size 25 \
  --min-ram 64 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script Meta_Coding_AoU.sh \
  --tasks score_task.txt

Job properties:
  job-id: meta-codin--williamsjacr--240911-212807-78
  job-name: meta-coding-aou
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/9957613312611743355
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/6784081199623067344
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/3520714611431293840
Launched job-id: meta-codin--williamsjacr--240911-212807-78
3 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'meta-codin--williamsjacr--240911-212807-78' --users 'williamsjacr' --status '*'
To cancel the job, run:
  ddel --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'meta-codin--williamsjacr--240911-212807-78' --users 'williamsjacr'
Waiting for job to complete...
Monitoring for failed tasks to retry...
*** This dsub process must

In [35]:
%%writefile Meta_Coding_AoU_LongMasks.R
rm(list=ls())
gc()

arrayid_longmask <- as.numeric(commandArgs(TRUE)[1])

Null_Model <- commandArgs(TRUE)[2]
print(Null_Model)

Annotation_name_catalog <- commandArgs(TRUE)[3]
print(Annotation_name_catalog)

GDS_File_Chr2 <- commandArgs(TRUE)[4]
print(GDS_File_Chr2)

GDS_File_Chr5 <- commandArgs(TRUE)[5]
print(GDS_File_Chr5)

OUTPUT_PATH <- commandArgs(TRUE)[6]
print(OUTPUT_PATH)

time <- system.time({
  ## load required packages
  library(gdsfmt)
  library(SeqArray)
  library(SeqVarTools)
  library(STAAR)
  library(STAARpipeline)
  library(readr)
  library(dplyr)
  library(stringr)
  
  library(devtools)
#  build(R_PACKAGE,path = "")
#  install.packages("MetaSTAAR_0.9.6.3.tar.gz", repos = NULL, type="source")
  library(MetaSTAAR)
  library(expm)
  library(Matrix)
  
  MetaSTAARlite_worker_sumstat <- function(genotype,obj_nullmodel,variant_info,qc_label=NULL,
                                           annotation_phred=NULL,for_individual_analysis=FALSE){
    
    if(!inherits(genotype, "matrix") && !inherits(genotype, "Matrix")){
      stop("genotype is not a matrix!")
    }
    
    if(inherits(genotype, "sparseMatrix")){
      genotype <- as.matrix(genotype)
    }
    
    if(dim(genotype)[2] != dim(variant_info)[1]){
      stop(paste0("Dimensions don't match for genotype and variant_info!"))
    }
    
    if(!is.null(qc_label) && dim(variant_info)[1] != length(qc_label)){
      stop(paste0("Dimensions don't match for variant_info and qc_label!"))
    }
    
    if(!is.null(annotation_phred) && dim(annotation_phred)[1] != dim(variant_info)[1]){
      stop(paste0("Dimensions don't match for annotation_phred and variant_info!"))
    }
    
    N <- dim(genotype)[1]
    alt_AC <- as.integer(colSums(2 - genotype))
    MAC <- as.integer(pmin(alt_AC, 2 * N - alt_AC))
    genotype <- matrix_flip(genotype)
    MAF <- genotype$MAF
    variant_label <- as.vector(MAF>0)
    Geno <- genotype$Geno[,variant_label,drop=FALSE]
    Geno <- as(Geno,"dgCMatrix")
    rm(genotype)
    gc()
    
    if(obj_nullmodel$relatedness){
      if(!obj_nullmodel$sparse_kins){
        stop(paste0("Please use a sparse genetic relatedness matrix when fitting the null model!"))
      }
    }else{
      if(obj_nullmodel$family[1] == "binomial"){
        obj_nullmodel$Sigma_i <- Diagonal(x = obj_nullmodel$weights)
      }else if(obj_nullmodel$family[1] == "gaussian"){
        obj_nullmodel$Sigma_i <- Diagonal(length(obj_nullmodel$y)) / summary(obj_nullmodel)$dispersion
      }
      obj_nullmodel$Sigma_iX <- obj_nullmodel$Sigma_i %*% model.matrix(obj_nullmodel)
      obj_nullmodel$cov <- solve(t(model.matrix(obj_nullmodel)) %*% obj_nullmodel$Sigma_i %*% model.matrix(obj_nullmodel))
      obj_nullmodel$scaled.residuals <- (obj_nullmodel$y - obj_nullmodel$fitted.values) / summary(obj_nullmodel)$dispersion
    }
    
    GTSinvX_cov <- as(t(Geno) %*% obj_nullmodel$Sigma_iX,"matrix") %*% sqrtm(obj_nullmodel$cov)
    #V <- diag(t(Geno) %*% obj_nullmodel$Sigma_i %*% Geno - GTSinvX_cov %*% t(GTSinvX_cov))
    V <- colSums(Geno * (obj_nullmodel$Sigma_i %*% Geno)) - rowSums(GTSinvX_cov^2) # faster for large number of variants
    U <- as.vector(t(Geno) %*% obj_nullmodel$scaled.residuals)
    rm(Geno)
    gc()
    
    if (!is.null(qc_label)){
      sumstat <- data.frame(variant_info[,c("chr","pos","ref","alt")],qc_label,alt_AC,MAC,MAF,N)
    }else{
      sumstat <- data.frame(variant_info[,c("chr","pos","ref","alt")],alt_AC,MAC,MAF,N)
    }
    if (!is.null(annotation_phred)){
      sumstat <- cbind(sumstat,annotation_phred)
    }
    sumstat <- sumstat[variant_label,]
    if(!for_individual_analysis){
      sumstat <- cbind(sumstat,U,V,GTSinvX_cov)
    }else{
      sumstat <- cbind(sumstat,U,V)
    }
    
    return(sumstat)
  }
  
  MetaSTAARlite_worker_cov <- function(genotype,obj_nullmodel,cov_maf_cutoff=0.05,
                                       qc_label=NULL,signif.digits=3){
    
    if(!inherits(genotype, "matrix") && !inherits(genotype, "Matrix")){
      stop("genotype is not a matrix!")
    }
    
    if(cov_maf_cutoff < 0 | cov_maf_cutoff > 0.5){
      stop("cov_maf_cutoff should be a number between 0 and 0.5!")
    }
    
    if(cov_maf_cutoff == 0.5){
      cov_maf_cutoff <- 0.5 + 1e-16
    }
    
    if(inherits(genotype, "sparseMatrix")){
      MAF <- colMeans(genotype)/2
      if (!is.null(qc_label)){
        RV_label <- as.vector((MAF<cov_maf_cutoff)&(MAF>0)&(qc_label=="PASS"))
      }else{
        RV_label <- as.vector((MAF<cov_maf_cutoff)&(MAF>0))
      }
      Geno_rare <- genotype[,RV_label,drop=FALSE]
      Geno_rare <- as(Geno_rare,"dgCMatrix")
    }else{
      genotype <- matrix_flip(genotype)
      MAF <- genotype$MAF
      if (!is.null(qc_label)){
        RV_label <- as.vector((MAF<cov_maf_cutoff)&(MAF>0)&(qc_label=="PASS"))
      }else{
        RV_label <- as.vector((MAF<cov_maf_cutoff)&(MAF>0))
      }
      Geno_rare <- genotype$Geno[,RV_label,drop=FALSE]
      Geno_rare <- as(Geno_rare,"dgCMatrix")
    }
    
    rm(genotype,MAF)
    gc()
    
    if(obj_nullmodel$relatedness){
      if(!obj_nullmodel$sparse_kins){
        stop(paste0("Please use a sparse genetic relatedness matrix when fitting the null model!"))
      }
    }else{
      if(obj_nullmodel$family[1] == "binomial"){
        obj_nullmodel$Sigma_i <- Diagonal(x = obj_nullmodel$weights)
      }else if(obj_nullmodel$family[1] == "gaussian"){
        obj_nullmodel$Sigma_i <- Diagonal(length(obj_nullmodel$y)) / summary(obj_nullmodel)$dispersion
      }
      obj_nullmodel$Sigma_iX <- obj_nullmodel$Sigma_i %*% model.matrix(obj_nullmodel)
      obj_nullmodel$cov <- solve(t(model.matrix(obj_nullmodel)) %*% obj_nullmodel$Sigma_i %*% model.matrix(obj_nullmodel))
      obj_nullmodel$scaled.residuals <- (obj_nullmodel$y - obj_nullmodel$fitted.values) / summary(obj_nullmodel)$dispersion
    }
    
    GTSinvG_rare <- t(obj_nullmodel$Sigma_i %*% Geno_rare) %*% Geno_rare
    rm(Geno_rare)
    gc()
    
    GTSinvG_rare <- as(GTSinvG_rare,"TsparseMatrix")
    remove_ind <- (GTSinvG_rare@j < GTSinvG_rare@i) # Be careful about multi-allelic issue
    GTSinvG_rare@i <- GTSinvG_rare@i[!remove_ind]
    GTSinvG_rare@j <- GTSinvG_rare@j[!remove_ind]
    GTSinvG_rare@x <- GTSinvG_rare@x[!remove_ind]
    rm(remove_ind)
    gc()
    GTSinvG_rare <- as(GTSinvG_rare,"CsparseMatrix")
    
    ### Create a version of GTSinvG_rare with rounded significant digits
    if(!is.null(signif.digits)){
      GTSinvG_rare <- signif(GTSinvG_rare, digits = signif.digits)
    }
    
    return(GTSinvG_rare)
  }
  
  
  coding_MetaSTAARlite_worker <- function(chr,gene_name,genofile,obj_nullmodel,genes,known_loci=NULL,
                                          cov_maf_cutoff=0.05,signif.digits=NULL,
                                          QC_label="annotation/filter",check_qc_label=FALSE,variant_type=c("SNV","Indel","variant"),
                                          Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                                          Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                                          silent=FALSE){
    
    ## evaluate choices
    variant_type <- match.arg(variant_type)
    
    phenotype.id <- obj_nullmodel$id_include
    
    filter <- seqGetData(genofile, QC_label)
    if(variant_type=="variant")
    {
      if(check_qc_label)
      {
        SNVlist <- TRUE
      }else
      {
        SNVlist <- filter == "PASS"
      }
    }
    
    if(variant_type=="SNV")
    {
      if(check_qc_label)
      {
        SNVlist <- isSNV(genofile)
      }else
      {
        SNVlist <- (filter == "PASS") & isSNV(genofile)
      }
    }
    
    if(variant_type=="Indel")
    {
      if(check_qc_label)
      {
        SNVlist <- !isSNV(genofile)
      }else
      {
        SNVlist <- (filter == "PASS") & (!isSNV(genofile))
      }
    }
    
    position <- as.numeric(seqGetData(genofile, "position"))
    variant.id <- seqGetData(genofile, "variant.id")
    
    if(!is.null(known_loci))
    {
      allele <- seqGetData(genofile, "allele")
      
      loc_SNV <- c()
      for (i in 1:dim(known_loci)[1]){
        loc_SNV <- c(loc_SNV,which((position==known_loci$POS[i])&(allele==paste0(known_loci$REF[i],",",known_loci$ALT[i]))))
      }
      
      seqSetFilter(genofile,variant.id=variant.id[loc_SNV],sample.id=phenotype.id)
      
      G_SNV <- seqGetData(genofile, "$dosage")
      
      if (!is.null(G_SNV)){
        id.SNV <- seqGetData(genofile,"sample.id")
        id.SNV.match <- rep(0,length(id.SNV))
        
        for(i in 1:length(id.SNV))
        {
          id.SNV.match[i] <- which.max(id.SNV==phenotype.id[i])
        }
        
        G_SNV <- G_SNV[id.SNV.match,,drop=FALSE]
        G_SNV_MAF <- apply(G_SNV, 2, function(x){sum(x[!is.na(x)])/2/sum(!is.na(x))})
        for (i in 1:length(G_SNV_MAF)){
          if (G_SNV_MAF[i] <= 0.5){
            G_SNV[is.na(G_SNV[,i]),i] <- 0
          }
          else{
            G_SNV[is.na(G_SNV[,i]),i] <- 2
          }
        }
        
        pos_adj <- as.integer(seqGetData(genofile, "position"))
        ref_adj <- as.character(seqGetData(genofile, "$ref"))
        alt_adj <- as.character(seqGetData(genofile, "$alt"))
        variant_adj_info <- data.frame(chr,pos_adj,ref_adj,alt_adj)
        colnames(variant_adj_info) <- c("chr","pos","ref","alt")
        variant_adj_info
        
        seqResetFilter(genofile)
      }
    }
    
    rm(filter)
    gc()
    
    ### Gene
    kk <- which(genes[,1]==gene_name)
    
    sub_start_loc <- genes[kk,3]
    sub_end_loc <- genes[kk,4]
    
    is.in <- (SNVlist)&(position>=sub_start_loc)&(position<=sub_end_loc)
    variant.id.gene <- variant.id[is.in]
    
    rm(position)
    gc()
    
    seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
    
    ## Gencode_Exonic
    GENCODE.EXONIC.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.EXONIC.Category")]))
    ## Gencode
    GENCODE.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Category")]))
    ## Meta.SVM.Pred
    MetaSVM_pred <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="MetaSVM")]))
    
    ################################################
    #           Coding
    ################################################
    variant.id.gene <- seqGetData(genofile, "variant.id")
    lof.in.coding <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|(GENCODE.Category=="ncRNA_splicing")|(GENCODE.Category=="ncRNA_exonic;splicing")|(GENCODE.EXONIC.Category=="nonsynonymous SNV")|(GENCODE.EXONIC.Category=="synonymous SNV")
    variant.id.gene <- variant.id.gene[lof.in.coding]
    
    seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
    
    ## Gencode_Exonic
    GENCODE.EXONIC.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.EXONIC.Category")]))
    ## Gencode
    GENCODE.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Category")]))
    ## Meta.SVM.Pred
    MetaSVM_pred <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="MetaSVM")]))
    
    ## Annotation
    Anno.Int.PHRED.sub <- NULL
    Anno.Int.PHRED.sub.name <- NULL
    
    if(variant_type=="SNV")
    {
      if(Use_annotation_weights)
      {
        for(k in 1:length(Annotation_name))
        {
          if(Annotation_name[k]%in%Annotation_name_catalog$name)
          {
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
            Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
            
            if(Annotation_name[k]=="CADD")
            {
              Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
            }
            Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
          }
        }
        
        Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
        colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
      }
    }
    
    ################################################
    #                  plof_ds
    ################################################
    variant.id.gene <- seqGetData(genofile, "variant.id")
    lof.in.plof <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|(GENCODE.Category=="ncRNA_splicing")|(GENCODE.Category=="ncRNA_exonic;splicing")|((GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D"))
    variant.id.gene.category <- variant.id.gene[lof.in.plof]
    
    seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
    
    pos <- as.integer(seqGetData(genofile, "position"))
    ref <- as.character(seqGetData(genofile, "$ref"))
    alt <- as.character(seqGetData(genofile, "$alt"))
    if(check_qc_label){
      qc_label <- as.character(seqGetData(genofile, QC_label))
    }else{
      qc_label <- NULL
    }
    
    ## genotype id
    id.genotype <- seqGetData(genofile,"sample.id")
    # id.genotype.match <- rep(0,length(id.genotype))
    
    id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
    phenotype.id.merge <- data.frame(phenotype.id)
    phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
    id.genotype.match <- phenotype.id.merge$index
    
    ## Genotype
    Geno <- seqGetData(genofile, "$dosage")
    Geno <- Geno[id.genotype.match,,drop=FALSE]
    
    summary_stat_list <- list()
    GTSinvG_rare_list <- list()
    cov_cond_list <- list()
    
    summary_stat <- NULL
    GTSinvG_rare <- NULL
    cov_cond <- NULL
    
    if(!is.null(Geno))
    {
      # Summary statistics
      genotype <- matrix_impute(Geno)
      variant_info <- data.frame(chr,pos,ref,alt,row.names=NULL)
      
      ## Annotation
      Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.plof,]
      
      try(summary_stat <- MetaSTAARlite_worker_sumstat(genotype,obj_nullmodel,variant_info,qc_label,
                                                       Anno.Int.PHRED.sub.category),silent=silent)
      
      # Covariance matrices
      genotype <- matrix_flip(Geno)$Geno
      genotype <- as(genotype,"dgCMatrix")
      
      rm(Geno)
      gc()
      
      try(GTSinvG_rare <- MetaSTAARlite_worker_cov(genotype,obj_nullmodel,cov_maf_cutoff,
                                                   qc_label,signif.digits),silent=silent)
      
      # Covariance matrices for conditional analysis
      if(!is.null(known_loci))
      {
        try(cov_cond <- MetaSTAARlite_worker_cov_cond(genotype,G_SNV,obj_nullmodel,variant_info,variant_adj_info),silent=silent)
      }
    }
    
    summary_stat_list[["plof_ds"]] <- summary_stat
    GTSinvG_rare_list[["plof_ds"]] <- GTSinvG_rare
    cov_cond_list[["plof_ds"]] <- cov_cond
    
    #####################################################
    #                      plof
    #####################################################
    lof.in.plof <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|(GENCODE.Category=="ncRNA_splicing")|(GENCODE.Category=="ncRNA_exonic;splicing")
    variant.id.gene.category <- variant.id.gene[lof.in.plof]
    
    seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
    
    pos <- as.integer(seqGetData(genofile, "position"))
    ref <- as.character(seqGetData(genofile, "$ref"))
    alt <- as.character(seqGetData(genofile, "$alt"))
    if(check_qc_label){
      qc_label <- as.character(seqGetData(genofile, QC_label))
    }else{
      qc_label <- NULL
    }
    
    ## genotype id
    id.genotype <- seqGetData(genofile,"sample.id")
    # id.genotype.match <- rep(0,length(id.genotype))
    
    id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
    phenotype.id.merge <- data.frame(phenotype.id)
    phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
    id.genotype.match <- phenotype.id.merge$index
    
    ## Genotype
    Geno <- seqGetData(genofile, "$dosage")
    Geno <- Geno[id.genotype.match,,drop=FALSE]
    
    summary_stat <- NULL
    GTSinvG_rare <- NULL
    cov_cond <- NULL
    
    if(!is.null(Geno))
    {
      # Summary statistics
      genotype <- matrix_impute(Geno)
      variant_info <- data.frame(chr,pos,ref,alt,row.names=NULL)
      
      ## Annotation
      Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.plof,]
      
      try(summary_stat <- MetaSTAARlite_worker_sumstat(genotype,obj_nullmodel,variant_info,qc_label,
                                                       Anno.Int.PHRED.sub.category),silent=silent)
      
      # Covariance matrices
      genotype <- matrix_flip(Geno)$Geno
      genotype <- as(genotype,"dgCMatrix")
      
      rm(Geno)
      gc()
      
      try(GTSinvG_rare <- MetaSTAARlite_worker_cov(genotype,obj_nullmodel,cov_maf_cutoff,
                                                   qc_label,signif.digits),silent=silent)
      
      # Covariance matrices for conditional analysis
      if(!is.null(known_loci))
      {
        try(cov_cond <- MetaSTAARlite_worker_cov_cond(genotype,G_SNV,obj_nullmodel,variant_info,variant_adj_info),silent=silent)
      }
    }
    
    summary_stat_list[["plof"]] <- summary_stat
    GTSinvG_rare_list[["plof"]] <- GTSinvG_rare
    cov_cond_list[["plof"]] <- cov_cond
    
    #############################################
    #             synonymous
    #############################################
    lof.in.synonymous <- (GENCODE.EXONIC.Category=="synonymous SNV")
    variant.id.gene.category <- variant.id.gene[lof.in.synonymous]
    
    seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
    
    pos <- as.integer(seqGetData(genofile, "position"))
    ref <- as.character(seqGetData(genofile, "$ref"))
    alt <- as.character(seqGetData(genofile, "$alt"))
    if(check_qc_label){
      qc_label <- as.character(seqGetData(genofile, QC_label))
    }else{
      qc_label <- NULL
    }
    
    ## genotype id
    id.genotype <- seqGetData(genofile,"sample.id")
    # id.genotype.match <- rep(0,length(id.genotype))
    
    id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
    phenotype.id.merge <- data.frame(phenotype.id)
    phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
    id.genotype.match <- phenotype.id.merge$index
    
    ## Genotype
    Geno <- seqGetData(genofile, "$dosage")
    Geno <- Geno[id.genotype.match,,drop=FALSE]
    
    summary_stat <- NULL
    GTSinvG_rare <- NULL
    cov_cond <- NULL
    
    if(!is.null(Geno))
    {
      # Summary statistics
      genotype <- matrix_impute(Geno)
      variant_info <- data.frame(chr,pos,ref,alt,row.names=NULL)
      
      ## Annotation
      Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.synonymous,]
      
      try(summary_stat <- MetaSTAARlite_worker_sumstat(genotype,obj_nullmodel,variant_info,qc_label,
                                                       Anno.Int.PHRED.sub.category),silent=silent)
      
      # Covariance matrices
      genotype <- matrix_flip(Geno)$Geno
      genotype <- as(genotype,"dgCMatrix")
      
      rm(Geno)
      gc()
      
      try(GTSinvG_rare <- MetaSTAARlite_worker_cov(genotype,obj_nullmodel,cov_maf_cutoff,
                                                   qc_label,signif.digits),silent=silent)
      
      # Covariance matrices for conditional analysis
      if(!is.null(known_loci))
      {
        try(cov_cond <- MetaSTAARlite_worker_cov_cond(genotype,G_SNV,obj_nullmodel,variant_info,variant_adj_info),silent=silent)
      }
    }
    
    summary_stat_list[["synonymous"]] <- summary_stat
    GTSinvG_rare_list[["synonymous"]] <- GTSinvG_rare
    cov_cond_list[["synonymous"]] <- cov_cond
    
    #################################################
    #        missense
    #################################################
    lof.in.missense <- (GENCODE.EXONIC.Category=="nonsynonymous SNV")
    variant.id.gene.category <- variant.id.gene[lof.in.missense]
    
    seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
    
    pos <- as.integer(seqGetData(genofile, "position"))
    ref <- as.character(seqGetData(genofile, "$ref"))
    alt <- as.character(seqGetData(genofile, "$alt"))
    if(check_qc_label){
      qc_label <- as.character(seqGetData(genofile, QC_label))
    }else{
      qc_label <- NULL
    }
    
    ## genotype id
    id.genotype <- seqGetData(genofile,"sample.id")
    # id.genotype.match <- rep(0,length(id.genotype))
    
    id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
    phenotype.id.merge <- data.frame(phenotype.id)
    phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
    id.genotype.match <- phenotype.id.merge$index
    
    ## Genotype
    Geno <- seqGetData(genofile, "$dosage")
    Geno <- Geno[id.genotype.match,,drop=FALSE]
    
    summary_stat <- NULL
    GTSinvG_rare <- NULL
    cov_cond <- NULL
    
    if(!is.null(Geno))
    {
      # Summary statistics
      genotype <- matrix_impute(Geno)
      variant_info <- data.frame(chr,pos,ref,alt,row.names=NULL)
      
      ## Annotation
      Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.missense,]
      
      try(summary_stat <- MetaSTAARlite_worker_sumstat(genotype,obj_nullmodel,variant_info,qc_label,
                                                       Anno.Int.PHRED.sub.category),silent=silent)
      
      # Covariance matrices
      genotype <- matrix_flip(Geno)$Geno
      genotype <- as(genotype,"dgCMatrix")
      
      rm(Geno)
      gc()
      
      try(GTSinvG_rare <- MetaSTAARlite_worker_cov(genotype,obj_nullmodel,cov_maf_cutoff,
                                                   qc_label,signif.digits),silent=silent)
      
      # Covariance matrices for conditional analysis
      if(!is.null(known_loci))
      {
        try(cov_cond <- MetaSTAARlite_worker_cov_cond(genotype,G_SNV,obj_nullmodel,variant_info,variant_adj_info),silent=silent)
      }
    }
    
    summary_stat_list[["missense"]] <- summary_stat
    GTSinvG_rare_list[["missense"]] <- GTSinvG_rare
    cov_cond_list[["missense"]] <- cov_cond
    
    #################################################
    #         disruptive missense
    #################################################
    lof.in.dmissense <- (GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D")
    variant.id.gene.category <- variant.id.gene[lof.in.dmissense]
    
    seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
    
    pos <- as.integer(seqGetData(genofile, "position"))
    ref <- as.character(seqGetData(genofile, "$ref"))
    alt <- as.character(seqGetData(genofile, "$alt"))
    if(check_qc_label){
      qc_label <- as.character(seqGetData(genofile, QC_label))
    }else{
      qc_label <- NULL
    }
    
    ## genotype id
    id.genotype <- seqGetData(genofile,"sample.id")
    # id.genotype.match <- rep(0,length(id.genotype))
    
    id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
    phenotype.id.merge <- data.frame(phenotype.id)
    phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
    id.genotype.match <- phenotype.id.merge$index
    
    ## Genotype
    Geno <- seqGetData(genofile, "$dosage")
    Geno <- Geno[id.genotype.match,,drop=FALSE]
    
    summary_stat <- NULL
    GTSinvG_rare <- NULL
    cov_cond <- NULL
    
    if(!is.null(Geno))
    {
      # Summary statistics
      genotype <- matrix_impute(Geno)
      variant_info <- data.frame(chr,pos,ref,alt,row.names=NULL)
      
      ## Annotation
      Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.dmissense,]
      
      try(summary_stat <- MetaSTAARlite_worker_sumstat(genotype,obj_nullmodel,variant_info,qc_label,
                                                       Anno.Int.PHRED.sub.category),silent=silent)
      
      # Covariance matrices
      genotype <- matrix_flip(Geno)$Geno
      genotype <- as(genotype,"dgCMatrix")
      
      rm(Geno)
      gc()
      
      try(GTSinvG_rare <- MetaSTAARlite_worker_cov(genotype,obj_nullmodel,cov_maf_cutoff,
                                                   qc_label,signif.digits),silent=silent)
      
      # Covariance matrices for conditional analysis
      if(!is.null(known_loci))
      {
        try(cov_cond <- MetaSTAARlite_worker_cov_cond(genotype,G_SNV,obj_nullmodel,variant_info,variant_adj_info),silent=silent)
      }
    }
    
    summary_stat_list[["disruptive_missense"]] <- summary_stat
    GTSinvG_rare_list[["disruptive_missense"]] <- GTSinvG_rare
    cov_cond_list[["disruptive_missense"]] <- cov_cond
    
    seqResetFilter(genofile)
    
    if(!is.null(known_loci))
    {
      return(list(summary_stat_list=summary_stat_list,
                  GTSinvG_rare_list=GTSinvG_rare_list,
                  cov_cond_list=cov_cond_list))
    }else
    {
      return(list(summary_stat_list=summary_stat_list,
                  GTSinvG_rare_list=GTSinvG_rare_list))
    }
  }
  
  
  ###########################################################
  #           User Input
  ###########################################################
  ## Null model
  obj_nullmodel <- get(load(Null_Model))

## Parameter
QC_label <- "annotation/filter"
geno_missing_imputation <- "mean"
variant_type <- "SNV"

## Annotation_dir
Annotation_dir <- "annotation/info/FunctionalAnnotation"
## Annotation channel
Annotation_name_catalog <- read.csv(Annotation_name_catalog)
## Use_annotation_weights
Use_annotation_weights <- TRUE
## Annotation name
Annotation_name <- c("CADD","LINSIGHT","FATHMM.XF","aPC.EpigeneticActive","aPC.EpigeneticRepressed","aPC.EpigeneticTranscription",
                     "aPC.Conservation","aPC.LocalDiversity","aPC.Mappability","aPC.TF","aPC.Protein")

agds_dir <- vector()
agds_dir[2] <- GDS_File_Chr2
agds_dir[5] <- GDS_File_Chr5
    
  ## output file name
  output_file_name <- paste0(OUTPUT_PATH,"/AoU_TC_Coding")
  
  ###########################################################
  #           Main Function 
  ###########################################################
  ## gene number in job
  gene_num_in_array <- 50
  group.num.allchr <- ceiling(table(genes_info[,2])/gene_num_in_array)
  sum(group.num.allchr)
  
  ## analyze large coding masks
  arrayid <- c(57,112,112,113,113,113,113,113,113,113)
  sub_seq_id <- c(840,543,544,575,576,577,578,579,580,582)
  
  region_spec <- data.frame(arrayid,sub_seq_id) 
  sub_seq_id <- ((arrayid_longmask-1)*5+1):min(arrayid_longmask*5,length(arrayid))
  
  genes <- genes_info
  
  coding_sumstat <- list()
  coding_cov <- list()
  for(kk in sub_seq_id){
    print(kk)
    arrayid <- region_spec$arrayid[kk]
    sub_id <- region_spec$sub_seq_id[kk]
    
    chr <- which.max(arrayid <= cumsum(group.num.allchr))
    
    ## aGDS file
    agds.path <- agds_dir[chr]
    genofile <- seqOpen(agds.path)
    
    genes_info_chr <- genes_info[genes_info[,2]==chr,]
    gene_name <- genes_info_chr[sub_id,1]
    results_temp <- coding_MetaSTAARlite_worker(chr=chr,gene_name=gene_name,genofile=genofile,obj_nullmodel=obj_nullmodel,genes=genes,
                                                cov_maf_cutoff=0.05,signif.digits=NULL,
                                                QC_label=QC_label,check_qc_label=TRUE,variant_type=variant_type,
                                                Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                                                Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name)
    coding_sumstat[[gene_name]] <- results_temp$summary_stat_list
    coding_cov[[gene_name]] <- results_temp$GTSinvG_rare_list
    
    seqClose(genofile)
  }
  
  save(coding_sumstat,file=paste0(output_file_name,"_sumstat_",arrayid_longmask+379,".Rdata"),compress = "xz")
  save(coding_cov,file=paste0(output_file_name,"_cov_",arrayid_longmask+379,".Rdata"),compress = "xz")
  
})[3]

save(time,file = paste0(output_file_name,"_time_",arrayid_longmask+379,".Rdata"))

Overwriting Meta_Coding_AoU_LongMasks.R


In [36]:
%%writefile Meta_Coding_AoU_LongMasks.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${arrayid} ${Null_Model} ${Annotation_name_catalog} ${GDS_File_Chr2} ${GDS_File_Chr5} ${OUTPUT_PATH}

Overwriting Meta_Coding_AoU_LongMasks.sh


In [40]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

for(arrayid in c(1,2)){
        tasks <- rbind(tasks, data.frame(
            '--env arrayid'=arrayid,
            '--input Null_Model'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/TC_Null_Model.RData",
            '--input Annotation_name_catalog'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataGDS/acaf_threshold_v7/Annotation_name_catalog.csv",
            '--input GDS_File_Chr2'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataAGDS/exome_v7.1/exome.chr",2,".gds"),
            '--input GDS_File_Chr5'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataAGDS/exome_v7.1/exome.chr",5,".gds"),
            '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/Meta_Coding_AoU_LongMasks.R",
            '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/AoU_Sumstats/",
            check.names = FALSE
        )) 
    }
  
write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [41]:
!Rscript score_task.R

In [42]:
!gsutil -m cp Meta_Coding_AoU_LongMasks.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/

Copying file://Meta_Coding_AoU_LongMasks.R [Content-Type=application/octet-stream]...
/ [1/1 files][ 28.3 KiB/ 28.3 KiB] 100% Done                                    
Operation completed over 1 objects/28.3 KiB.                                     


In [43]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 100 \
  --boot-disk-size 25 \
  --min-ram 128 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script Meta_Coding_AoU_LongMasks.sh \
  --tasks score_task.txt

Job properties:
  job-id: meta-codin--williamsjacr--241118-144831-35
  job-name: meta-coding-aou-longmasks
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/15715809343684227583
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/14475785995333795255
Launched job-id: meta-codin--williamsjacr--241118-144831-35
2 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'meta-codin--williamsjacr--241118-144831-35' --users 'williamsjacr' --status '*'
To cancel the job, run:
  ddel --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'meta-codin--williamsjacr--241118-144831-35' --users 'williamsjacr'


In [45]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'meta-codin--williamsjacr--241118-144831-35' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
meta-coding-...       2  Success   11-18 15:15:40
meta-coding-...       1  Success   11-18 15:33:54



In [7]:
%%writefile Meta_Coding_UKB_AoU.R
rm(list=ls())
gc()

arrayid <- as.numeric(commandArgs(TRUE)[1])

UKB_Sumstats <- commandArgs(TRUE)[2]
print(UKB_Sumstats)

AoU_Sumstats <- commandArgs(TRUE)[3]
print(AoU_Sumstats)

OUTPUT_PATH <- commandArgs(TRUE)[4]
print(OUTPUT_PATH)

time <- system.time({
    ## load required packages
library(STAAR)
library(STAARpipeline)
library(MetaSTAAR)
library(dplyr)
library(Matrix)

MetaSTAARlite_merge <- function(chr,sample.sizes,sumstat.list,cov.list,
                                rare_maf_cutoff=0.01,cov_maf_cutoff,
                                check_qc_label=FALSE,variant_type=c("SNV","Indel","variant"),
                                Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL){
  
  ## evaluate choices
  variant_type <- match.arg(variant_type)
  
  cov_maf_cutoff[cov_maf_cutoff == 0.5] <- 0.5 + 1e-16
  ### summary statistics
  sumstat.list <- lapply(sumstat.list, function(x) {
    if (!(is.null(x)) && !("qc_label" %in% colnames(x))){
      data.frame(x[,1:4],qc_label="PASS",x[,5:dim(x)[2]],stringsAsFactors = FALSE)
    }else{
      x
    }
  })
  sumstat.varid.list <- mapply(function(x,y) {
    x[(x$MAF<y)&(x$MAF>0)&(x$qc_label=="PASS"),1:4]
  }, x = sumstat.list, y = cov_maf_cutoff, SIMPLIFY = FALSE)
  sumstat.varid.merge <- do.call("rbind",sumstat.varid.list)
  sumstat.varid.nodup <- sumstat.varid.merge[!duplicated(sumstat.varid.merge),]
  if (is.null(sumstat.varid.nodup)) {
    return(NULL)
  }else if (dim(sumstat.varid.nodup)[1] == 0) {
    return(NULL)
  }
  sumstat.merge.list <- lapply(sumstat.list, function(x) {
    if (is.null(x)) {
      cbind(sumstat.varid.nodup,qc_label=NA,alt_AC=NA,MAC=NA,MAF=NA,N=NA,U=NA,V=NA,"1"=NA)
    }else {
      left_join(sumstat.varid.nodup,x,by=c("chr"="chr",
                                           "pos"="pos",
                                           "ref"="ref",
                                           "alt"="alt"))
    }
  })
  sumstat.merge.list <- mapply(function(x,y) {
    x[is.na(x[,"N"]),"N"] <- y
    x[is.na(x[,"qc_label"]),"qc_label"] <- "PASS"
    x[is.na(x)] <- 0
    return(x)
  }, x = sumstat.merge.list, y = sample.sizes, SIMPLIFY = FALSE)
  sumstat.varid.nodup$index <- 1:dim(sumstat.varid.nodup)[1]
  sumstat.index.list <- lapply(sumstat.varid.list, function(x) {
    if (is.null(x)) {
      integer(0)
    }else{
      left_join(x,sumstat.varid.nodup,by=c("chr"="chr",
                                           "pos"="pos",
                                           "ref"="ref",
                                           "alt"="alt"))$index
    }
  })
  rm(sumstat.list,sumstat.varid.list,sumstat.varid.merge)
  gc()
  
  ### covariance matrices
  cov.list <- lapply(cov.list, function(x) {
    if (is.null(x)) {
      as(matrix(nrow=0,ncol=0),"dgCMatrix")
    }else if (dim(x)[1] == 0) {
      as(matrix(nrow=0,ncol=0),"dgCMatrix")
    }else {
      x[,1:dim(x)[1]]
    }
  })
  
  cov.null <- matrix(0, nrow = dim(sumstat.varid.nodup)[1], ncol = dim(sumstat.varid.nodup)[1])
  cov.merge.list <- mapply(function(x,y) {
    if (length(x) == 1) {
      # in this case y is a scalar
      cov.null[x,x] <- y
    }else if (length(x) > 1) {
      cov.null[x,x] <- as.matrix(forceSymmetric(y, uplo = "U"))
    }
    return(cov.null)
  }, x = sumstat.index.list, y = cov.list, SIMPLIFY = FALSE)
  
  ### select rare variant based on the input cutoff
  alt_AC.merge <- as.integer(Reduce("+",lapply(sumstat.merge.list, function(x) {x$alt_AC})))
  N.merge.nonzero <- Reduce("+",lapply(sumstat.merge.list, function(x) {x$N * (x$MAC != 0)}))
  alt_AF.merge.nonzero <- alt_AC.merge / (2 * N.merge.nonzero)
  N.merge.zero <- Reduce("+",lapply(sumstat.merge.list, function(x) {x$N * (x$MAC == 0)}))
  alt_AC.merge <- alt_AC.merge + (alt_AF.merge.nonzero > 0.5) * (2 * N.merge.zero)
  N.merge <- Reduce("+",lapply(sumstat.merge.list, function(x) {x$N}))
  MAC.merge <- pmin(alt_AC.merge, 2 * N.merge - alt_AC.merge)
  MAF.merge <- MAC.merge / (2 * N.merge)
  rv.index <- (MAF.merge<rare_maf_cutoff) & Reduce("*",mapply(function(x,y) {
    x$MAF<y
  }, x = sumstat.merge.list, y = cov_maf_cutoff, SIMPLIFY = FALSE))
  if (check_qc_label){
    rv.index <- rv.index & Reduce("*",lapply(sumstat.merge.list, function(x) {
      x$qc_label=="PASS"
    }))
  }
  
  info <- cbind(sumstat.varid.nodup[,c("chr","pos","ref","alt")],
                MAC=MAC.merge,MAF=MAF.merge)[rv.index,]
  
  U.merge <- Reduce("+", lapply(sumstat.merge.list, function(x) {
    x <- x[rv.index,]
    return(x$U)
  }))
  
  cov.merge <- Reduce("+", mapply(function(x,y) {
    (x - as.matrix(y[,(which(colnames(y)=="V")+1):dim(y)[2]]) %*% t(as.matrix(y[,(which(colnames(y)=="V")+1):dim(y)[2]])))[rv.index, rv.index]
  }, x = cov.merge.list, y = sumstat.merge.list, SIMPLIFY = FALSE))
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV")
  {
    if(Use_annotation_weights)
    {
      for(k in 1:length(Annotation_name))
      {
        Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
        Annotation.list <- lapply(sumstat.merge.list, function(x) {
          if(Annotation_name[k]%in%colnames(x)) {
            x[[Annotation_name[k]]]
          }else {
            0
          }
        })
        Annotation.PHRED.sum <- Reduce("+", Annotation.list)
        Annotation.PHRED.gt0 <- Reduce("+",lapply(Annotation.list, function(x) {x > 0}))
        Annotation.PHRED <- Annotation.PHRED.sum / Annotation.PHRED.gt0
        Annotation.PHRED[!is.finite(Annotation.PHRED)] <- 0
        
        if(Annotation_name[k]=="aPC.LocalDiversity")
        {
          Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
          Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
        }
        Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- c(Anno.Int.PHRED.sub.name)
    }
  }
  
  annotation_phred <- Anno.Int.PHRED.sub[rv.index,]
  
  rm(list=setdiff(ls(), c("info","U.merge","cov.merge","annotation_phred")))
  gc()
  
  return(list(info=info,
              U=U.merge,
              cov=as.matrix(cov.merge),
              annotation_phred=annotation_phred))
}

coding_MetaSTAARlite <- function(chr,gene_name,genes,
                                 sample.sizes,coding_sumstat_gene_list,coding_cov_gene_list,
                                 cov_maf_cutoff,rare_maf_cutoff=0.01,rv_num_cutoff=2,
                                 check_qc_label=FALSE,variant_type=c("SNV","Indel","variant"),
                                 Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,silent=FALSE){

  ## evaluate choices
  variant_type <- match.arg(variant_type)

  ### Gene
  kk <- which(genes[,1]==gene_name)

  ################################################
  #                  plof_ds
  ################################################
  results_plof_ds <- c()

  sumstat.list <- lapply(coding_sumstat_gene_list, function(x) {
    x[["plof_ds"]]
  })
  cov.list <- lapply(coding_cov_gene_list, function(x) {
    x[["plof_ds"]]
  })

  gene_test_merge <- MetaSTAARlite_merge(chr,sample.sizes,sumstat.list,cov.list,
                                         cov_maf_cutoff=cov_maf_cutoff,rare_maf_cutoff=rare_maf_cutoff,
                                         check_qc_label=check_qc_label,variant_type=variant_type,
                                         Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name)
  if(length(gene_test_merge$U)>=2)
  {
    ## Annotation
    annotation_phred <- gene_test_merge$annotation_phred
    pvalues <- 0
    try(pvalues <- MetaSTAAR(gene_test_merge,annotation_phred,rv_num_cutoff),silent=silent)

    if(inherits(pvalues,"list"))
    {
      results_temp <- as.vector(genes[kk,])
      results_temp[3] <- "plof_ds"
      results_temp[2] <- chr
      results_temp[1] <- as.character(genes[kk,1])
      results_temp[4] <- pvalues$num_variant


      results_temp <- c(results_temp,pvalues$results_MetaSTAAR_S_1_25,pvalues$results_MetaSTAAR_S_1_1,
                        pvalues$results_MetaSTAAR_B_1_25,pvalues$results_MetaSTAAR_B_1_1,pvalues$results_MetaSTAAR_A_1_25,
                        pvalues$results_MetaSTAAR_A_1_1,pvalues$results_ACAT_O_MS,pvalues$results_MetaSTAAR_O)

      results_plof_ds <- rbind(results_plof_ds,results_temp)
    }
  }

  if(!is.null(results_plof_ds))
  {
    colnames(results_plof_ds) <- colnames(results_plof_ds, do.NULL = FALSE, prefix = "col")
    colnames(results_plof_ds)[1:4] <- c("Gene name","Chr","Category","#SNV")
    colnames(results_plof_ds)[(dim(results_plof_ds)[2]-1):dim(results_plof_ds)[2]] <- c("ACAT-O-MS","MetaSTAAR-O")
  }

  #####################################################
  #                      plof
  #####################################################
  results_plof <- c()

  sumstat.list <- lapply(coding_sumstat_gene_list, function(x) {
    x[["plof"]]
  })
  cov.list <- lapply(coding_cov_gene_list, function(x) {
    x[["plof"]]
  })

  gene_test_merge <- MetaSTAARlite_merge(chr,sample.sizes,sumstat.list,cov.list,
                                         cov_maf_cutoff=cov_maf_cutoff,rare_maf_cutoff=rare_maf_cutoff,
                                         check_qc_label=check_qc_label,variant_type=variant_type,
                                         Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name)
  if(length(gene_test_merge$U)>=2)
  {
    ## Annotation
    annotation_phred <- gene_test_merge$annotation_phred
    pvalues <- 0
    try(pvalues <- MetaSTAAR(gene_test_merge,annotation_phred,rv_num_cutoff),silent=silent)

    if(inherits(pvalues,"list"))
    {
      results_temp <- as.vector(genes[kk,])
      results_temp[3] <- "plof"
      results_temp[2] <- chr
      results_temp[1] <- as.character(genes[kk,1])
      results_temp[4] <- pvalues$num_variant


      results_temp <- c(results_temp,pvalues$results_MetaSTAAR_S_1_25,pvalues$results_MetaSTAAR_S_1_1,
                        pvalues$results_MetaSTAAR_B_1_25,pvalues$results_MetaSTAAR_B_1_1,pvalues$results_MetaSTAAR_A_1_25,
                        pvalues$results_MetaSTAAR_A_1_1,pvalues$results_ACAT_O_MS,pvalues$results_MetaSTAAR_O)

      results_plof <- rbind(results_plof,results_temp)
    }
  }

  if(!is.null(results_plof))
  {
    colnames(results_plof) <- colnames(results_plof, do.NULL = FALSE, prefix = "col")
    colnames(results_plof)[1:4] <- c("Gene name","Chr","Category","#SNV")
    colnames(results_plof)[(dim(results_plof)[2]-1):dim(results_plof)[2]] <- c("ACAT-O-MS","MetaSTAAR-O")
  }

  #############################################
  #             synonymous
  #############################################
  results_synonymous <- c()

  sumstat.list <- lapply(coding_sumstat_gene_list, function(x) {
    x[["synonymous"]]
  })
  cov.list <- lapply(coding_cov_gene_list, function(x) {
    x[["synonymous"]]
  })

  gene_test_merge <- MetaSTAARlite_merge(chr,sample.sizes,sumstat.list,cov.list,
                                         cov_maf_cutoff=cov_maf_cutoff,rare_maf_cutoff=rare_maf_cutoff,
                                         check_qc_label=check_qc_label,variant_type=variant_type,
                                         Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name)
  if(length(gene_test_merge$U)>=2)
  {
    ## Annotation
    annotation_phred <- gene_test_merge$annotation_phred
    pvalues <- 0
    try(pvalues <- MetaSTAAR(gene_test_merge,annotation_phred,rv_num_cutoff),silent=silent)

    if(inherits(pvalues,"list"))
    {
      results_temp <- as.vector(genes[kk,])
      results_temp[3] <- "synonymous"
      results_temp[2] <- chr
      results_temp[1] <- as.character(genes[kk,1])
      results_temp[4] <- pvalues$num_variant


      results_temp <- c(results_temp,pvalues$results_MetaSTAAR_S_1_25,pvalues$results_MetaSTAAR_S_1_1,
                        pvalues$results_MetaSTAAR_B_1_25,pvalues$results_MetaSTAAR_B_1_1,pvalues$results_MetaSTAAR_A_1_25,
                        pvalues$results_MetaSTAAR_A_1_1,pvalues$results_ACAT_O_MS,pvalues$results_MetaSTAAR_O)

      results_synonymous <- rbind(results_synonymous,results_temp)
    }
  }

  if(!is.null(results_synonymous))
  {
    colnames(results_synonymous) <- colnames(results_synonymous, do.NULL = FALSE, prefix = "col")
    colnames(results_synonymous)[1:4] <- c("Gene name","Chr","Category","#SNV")
    colnames(results_synonymous)[(dim(results_synonymous)[2]-1):dim(results_synonymous)[2]] <- c("ACAT-O-MS","MetaSTAAR-O")
  }

  #################################################
  #        missense
  #################################################
  results <- c()

  sumstat.list <- lapply(coding_sumstat_gene_list, function(x) {
    x[["missense"]]
  })
  cov.list <- lapply(coding_cov_gene_list, function(x) {
    x[["missense"]]
  })

  gene_test_merge <- MetaSTAARlite_merge(chr,sample.sizes,sumstat.list,cov.list,
                                         cov_maf_cutoff=cov_maf_cutoff,rare_maf_cutoff=rare_maf_cutoff,
                                         check_qc_label=check_qc_label,variant_type=variant_type,
                                         Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name)
  if(length(gene_test_merge$U)>=2)
  {
    ## Annotation
    annotation_phred <- gene_test_merge$annotation_phred
    pvalues <- 0
    try(pvalues <- MetaSTAAR(gene_test_merge,annotation_phred,rv_num_cutoff),silent=silent)

    if(inherits(pvalues,"list"))
    {
      results_temp <- as.vector(genes[kk,])
      results_temp[3] <- "missense"
      results_temp[2] <- chr
      results_temp[1] <- as.character(genes[kk,1])
      results_temp[4] <- pvalues$num_variant


      results_temp <- c(results_temp,pvalues$results_MetaSTAAR_S_1_25,pvalues$results_MetaSTAAR_S_1_1,
                        pvalues$results_MetaSTAAR_B_1_25,pvalues$results_MetaSTAAR_B_1_1,pvalues$results_MetaSTAAR_A_1_25,
                        pvalues$results_MetaSTAAR_A_1_1,pvalues$results_ACAT_O_MS,pvalues$results_MetaSTAAR_O)

      results <- rbind(results,results_temp)
    }
  }

  #################################################
  #         disruptive missense
  #################################################
  sumstat.list <- lapply(coding_sumstat_gene_list, function(x) {
    x[["disruptive_missense"]]
  })
  cov.list <- lapply(coding_cov_gene_list, function(x) {
    x[["disruptive_missense"]]
  })

  gene_test_merge <- MetaSTAARlite_merge(chr,sample.sizes,sumstat.list,cov.list,
                                         cov_maf_cutoff=cov_maf_cutoff,rare_maf_cutoff=rare_maf_cutoff,
                                         check_qc_label=check_qc_label,variant_type=variant_type,
                                         Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name)
  if(length(gene_test_merge$U)>=2)
  {
    ## Annotation
    annotation_phred <- gene_test_merge$annotation_phred
    pvalues <- 0
    try(pvalues <- MetaSTAAR(gene_test_merge,annotation_phred,rv_num_cutoff),silent=silent)

    if(inherits(pvalues,"list"))
    {
      results_temp <- as.vector(genes[kk,])
      results_temp[3] <- "disruptive_missense"
      results_temp[2] <- chr
      results_temp[1] <- as.character(genes[kk,1])
      results_temp[4] <- pvalues$num_variant


      results_temp <- c(results_temp,pvalues$results_MetaSTAAR_S_1_25,pvalues$results_MetaSTAAR_S_1_1,
                        pvalues$results_MetaSTAAR_B_1_25,pvalues$results_MetaSTAAR_B_1_1,pvalues$results_MetaSTAAR_A_1_25,
                        pvalues$results_MetaSTAAR_A_1_1,pvalues$results_ACAT_O_MS,pvalues$results_MetaSTAAR_O)

      results <- rbind(results,results_temp)
    }
  }

  if(!is.null(results))
  {
    colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
    colnames(results)[1:4] <- c("Gene name","Chr","Category","#SNV")
    colnames(results)[(dim(results)[2]-1):dim(results)[2]] <- c("ACAT-O-MS","MetaSTAAR-O")

    if(dim(results)[1]==1)
    {
      if(results[3]!="disruptive_missense")
      {
        results <- cbind(results,matrix(1,1,6))
        colnames(results)[(dim(results)[2]-5):dim(results)[2]] <- c("SKAT-MS(1,25)-Disruptive","SKAT-MS(1,1)-Disruptive","Burden-MS(1,25)-Disruptive","Burden-MS(1,1)-Disruptive","ACAT-V-MS(1,25)-Disruptive","ACAT-V-MS(1,1)-Disruptive")
        results_missense <- results
        results_ds <- c()
      }else
      {
        results_missense <- c()
        results_ds <- results
        results <- c()
      }
    }

    if(!is.null(results))
    {
      if(dim(results)[1]==2)
      {
        results_m <- c(results[1,],rep(0,6))
        names(results_m)[(length(results_m)-5):length(results_m)] <- c("SKAT-MS(1,25)-Disruptive","SKAT-MS(1,1)-Disruptive","Burden-MS(1,25)-Disruptive","Burden-MS(1,1)-Disruptive","ACAT-V-MS(1,25)-Disruptive","ACAT-V-MS(1,1)-Disruptive")
        results_m[(length(results_m)-5):length(results_m)] <- results[2,c("SKAT-MS(1,25)","SKAT-MS(1,1)","Burden-MS(1,25)","Burden-MS(1,1)","ACAT-V-MS(1,25)","ACAT-V-MS(1,1)")]
        apc_num <- (length(results_m)-18)/6
        p_seq <- c(1:apc_num,1:apc_num+(apc_num+1),1:apc_num+2*(apc_num+1),1:apc_num+3*(apc_num+1),1:apc_num+4*(apc_num+1),1:apc_num+5*(apc_num+1),(6*apc_num+9):(6*apc_num+14))
        results_m["MetaSTAAR-O"] <- CCT(as.numeric(results_m[5:length(results_m)][p_seq]))
        results_m["MetaSTAAR-S(1,25)"] <- CCT(as.numeric(results_m[5:length(results_m)][c(1:apc_num,6*apc_num+9)]))
        results_m["MetaSTAAR-S(1,1)"] <- CCT(as.numeric(results_m[5:length(results_m)][c(1:apc_num+(apc_num+1),6*apc_num+10)]))
        results_m["MetaSTAAR-B(1,25)"] <- CCT(as.numeric(results_m[5:length(results_m)][c(1:apc_num+2*(apc_num+1),6*apc_num+11)]))
        results_m["MetaSTAAR-B(1,1)"] <- CCT(as.numeric(results_m[5:length(results_m)][c(1:apc_num+3*(apc_num+1),6*apc_num+12)]))
        results_m["MetaSTAAR-A(1,25)"] <- CCT(as.numeric(results_m[5:length(results_m)][c(1:apc_num+4*(apc_num+1),6*apc_num+13)]))
        results_m["MetaSTAAR-A(1,1)"] <- CCT(as.numeric(results_m[5:length(results_m)][c(1:apc_num+5*(apc_num+1),6*apc_num+14)]))

        results_ds <- c()
        results_ds <- rbind(results_ds,results[2,])

        results <- c()
        results <- rbind(results,results_m)
      }
    }
  }else
  {
    results <- c()
    results_ds <- c()
  }

  results_coding <- list(plof=results_plof,plof_ds=results_plof_ds,missense=results,disruptive_missense=results_ds,synonymous=results_synonymous)

  return(results_coding)
}

###########################################################
#           User Input
###########################################################
## Directories of the study-specific summary statistics file folders
file.dir <- c(paste0(UKB_Sumstats,"/"),
              paste0(AoU_Sumstats,"/"))
file.prefix <- c("UKB_TC_Coding","AoU_TC_Coding")
## Sample sizes of participating studies
sample.sizes <- c(446933,94532)

## variant_type
variant_type <- "SNV"
## cov_maf_cutoff
cov_maf_cutoff <- c(0.05,0.05)

## Use_annotation_weights
Use_annotation_weights <- TRUE
## Annotation name
Annotation_name <- c("CADD","LINSIGHT","FATHMM.XF","aPC.EpigeneticActive","aPC.EpigeneticRepressed","aPC.EpigeneticTranscription",
                     "aPC.Conservation","aPC.LocalDiversity","aPC.Mappability","aPC.TF","aPC.Protein")

## output path
output_path <- OUTPUT_PATH
## output file name
output_file_name <- "UKB_AoU_Coding"

###########################################################
#           Main Function 
###########################################################
## gene number in job
gene_num_in_array <- 50
group.num.allchr <- ceiling(table(genes_info[,2])/gene_num_in_array)
sum(group.num.allchr)

chr <- as.integer(which.max(arrayid <= cumsum(group.num.allchr)))
group.num <- group.num.allchr[chr]

if (chr == 1){
  groupid <- arrayid
}else{
  groupid <- arrayid - cumsum(group.num.allchr)[chr-1]
}

genes_info_chr <- genes_info[genes_info[,2]==chr,]
sub_seq_num <- dim(genes_info_chr)[1]

if(groupid < group.num)
{
  sub_seq_id <- ((groupid - 1)*gene_num_in_array + 1):(groupid*gene_num_in_array)
}else
{
  sub_seq_id <- ((groupid - 1)*gene_num_in_array + 1):sub_seq_num
}

## exclude large coding masks
if(arrayid==57)
{
  sub_seq_id <- setdiff(sub_seq_id,840)
}

if(arrayid==112)
{
  sub_seq_id <- setdiff(sub_seq_id,c(543,544))
}

if(arrayid==113)
{
  sub_seq_id <- setdiff(sub_seq_id,c(575,576,577,578,579,580,582))
}

genes <- genes_info

results_coding <- c()

UKB_SumStat.File.Path <- paste0(file.dir[1],file.prefix[1],"_sumstat_",arrayid,"_470.Rdata")
AoU_SumStat.File.Path <- paste0(file.dir[2],file.prefix[2],"_sumstat_",arrayid,".Rdata")

UKB_Cov.File.Path <- paste0(file.dir[1],file.prefix[1],"_cov_",arrayid,"_470.Rdata")
AoU_Cov.File.Path <- paste0(file.dir[2],file.prefix[2],"_cov_",arrayid,".Rdata")

sumstat.file.path <- c(UKB_SumStat.File.Path,AoU_SumStat.File.Path)
cov.file.path <- c(UKB_Cov.File.Path,AoU_Cov.File.Path)
coding_sumstat_list <- sapply(sumstat.file.path, function(x) mget(load(x)), simplify = TRUE)
coding_cov_list <- sapply(cov.file.path, function(x) mget(load(x)), simplify = TRUE)

for(kk in sub_seq_id)
{
  print(kk)
  gene_name <- genes_info_chr[kk,1]
  coding_sumstat_gene_list <- lapply(sumstat.file.path, function(x) {
    coding_sumstat_list[[paste0(x,".coding_sumstat")]][[gene_name]]
  })
  coding_cov_gene_list <- lapply(cov.file.path, function(x) {
    coding_cov_list[[paste0(x,".coding_cov")]][[gene_name]]
  })
  results <- coding_MetaSTAARlite(chr=chr,gene_name=gene_name,genes=genes,
                                  sample.sizes=sample.sizes,coding_sumstat_gene_list=coding_sumstat_gene_list,
                                  coding_cov_gene_list=coding_cov_gene_list,
                                  cov_maf_cutoff=cov_maf_cutoff,
                                  rare_maf_cutoff=0.01,rv_num_cutoff=2,
                                  check_qc_label=TRUE,variant_type=variant_type,
                                  Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name)
  results_coding <- append(results_coding,results)
}

save(results_coding,file=paste0(output_path,"/",output_file_name,"_",arrayid,".Rdata"))
})[3]

save(time,file = paste0(output_path,"/",output_file_name,"_time_",arrayid,".Rdata"))

Writing Meta_Coding_UKB_AoU.R


In [8]:
%%writefile Meta_Coding_UKB_AoU.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${arrayid} ${UKB_Sumstats} ${AoU_Sumstats} ${OUTPUT_PATH}

Writing Meta_Coding_UKB_AoU.sh


In [11]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

for(arrayid in c(380:381)){
        tasks <- rbind(tasks, data.frame(
            '--env arrayid'=arrayid,
            '--input-recursive UKB_Sumstats'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/UKB_Sumstats",
            '--input-recursive AoU_Sumstats'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/AoU_Sumstats",
            '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/Meta_Coding_UKB_AoU.R",
            '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/UKB_AoU_Meta",
            check.names = FALSE
        )) 
    }
  
write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [12]:
!Rscript score_task.R

In [13]:
!gsutil -m cp Meta_Coding_UKB_AoU.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/

Copying file://Meta_Coding_UKB_AoU.R [Content-Type=application/octet-stream]...
/ [1/1 files][ 22.0 KiB/ 22.0 KiB] 100% Done                                    
Operation completed over 1 objects/22.0 KiB.                                     


In [14]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 100 \
  --boot-disk-size 25 \
  --min-ram 16 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script Meta_Coding_UKB_AoU.sh \
  --tasks score_task.txt

Job properties:
  job-id: meta-codin--williamsjacr--241120-151534-74
  job-name: meta-coding-ukb-aou
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/15867520956650507802
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/1469970834848913314
Launched job-id: meta-codin--williamsjacr--241120-151534-74
2 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'meta-codin--williamsjacr--241120-151534-74' --users 'williamsjacr' --status '*'
To cancel the job, run:
  ddel --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'meta-codin--williamsjacr--241120-151534-74' --users 'williamsjacr'


In [19]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'meta-codin--williamsjacr--241120-151534-74' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
meta-coding-...       2  Success   11-20 15:20:41
meta-coding-...       1  Success   11-20 15:20:26



In [26]:
%%writefile Meta_Coding_UKB_AoU_Summary.R

rm(list=ls())
gc()

Meta_SumStats <- commandArgs(TRUE)[1]
print(Meta_SumStats)

OUTPUT_PATH <- commandArgs(TRUE)[2]
print(OUTPUT_PATH)

## load required packages
library(STAAR)
library(STAARpipeline)
library(STAARpipelineSummary)
library(MetaSTAAR)

Gene_Centric_Coding_Results_Summary_meta <- function(gene_centric_coding_jobs_num,input_path,output_path,gene_centric_results_name,
                                                     alpha=2.5E-06,manhattan_plot=FALSE,QQ_plot=FALSE){

  #######################################################
  #     summarize unconditional analysis results
  #######################################################

  results_coding_genome <- c()

  for(kk in 1:gene_centric_coding_jobs_num)
  {
    print(kk)
    results_coding <- get(load(paste0(input_path,gene_centric_results_name,"_",kk,".Rdata")))

    results_coding_genome <- c(results_coding_genome,results_coding)
  }

  results_plof_genome <- c()
  results_plof_ds_genome <- c()
  results_missense_genome <- c()
  results_disruptive_missense_genome <- c()
  results_synonymous_genome <- c()

  for(kk in 1:length(results_coding_genome))
  {
    results <- results_coding_genome[[kk]]

    if(is.null(results)==FALSE)
    {
      ### plof
      if(results[3]=="plof")
      {
        results_plof_genome <- rbind(results_plof_genome,results)
      }
      ### plof_ds
      if(results[3]=="plof_ds")
      {
        results_plof_ds_genome <- rbind(results_plof_ds_genome,results)
      }
      ### missense
      if(results[3]=="missense")
      {
        results_missense_genome <- rbind(results_missense_genome,results)
      }
      ### disruptive_missense
      if(results[3]=="disruptive_missense")
      {
        results_disruptive_missense_genome <- rbind(results_disruptive_missense_genome,results)
      }
      ### synonymous
      if(results[3]=="synonymous")
      {
        results_synonymous_genome <- rbind(results_synonymous_genome,results)
      }
    }

    if(kk%%1000==0)
    {
      print(kk)
    }
  }
  
  ###### whole-genome results
  # plof
  save(results_plof_genome,file=paste0(output_path,"plof.Rdata"))
  # plof + disruptive missense
  save(results_plof_ds_genome,file=paste0(output_path,"plof_ds.Rdata"))
  # missense
  save(results_missense_genome,file=paste0(output_path,"missense.Rdata"))
  # disruptive missense
  save(results_disruptive_missense_genome,file=paste0(output_path,"disruptive_missense.Rdata"))
  # synonymous
  save(results_synonymous_genome,file=paste0(output_path,"synonymous.Rdata"))

  ###### significant results
  # plof
  plof_sig <- results_plof_genome[results_plof_genome[,"MetaSTAAR-O"]<alpha,,drop=FALSE]
  write.csv(plof_sig,file=paste0(output_path,"plof_sig.csv"))
  # missense
  missense_sig <- results_missense_genome[results_missense_genome[,"MetaSTAAR-O"]<alpha,,drop=FALSE]
  write.csv(missense_sig,file=paste0(output_path,"missense_sig.csv"))
  # synonymous
  synonymous_sig <- results_synonymous_genome[results_synonymous_genome[,"MetaSTAAR-O"]<alpha,,drop=FALSE]
  write.csv(synonymous_sig,file=paste0(output_path,"synonymous_sig.csv"))
  # plof_ds
  plof_ds_sig <- results_plof_ds_genome[results_plof_ds_genome[,"MetaSTAAR-O"]<alpha,,drop=FALSE]
  write.csv(plof_ds_sig,file=paste0(output_path,"plof_ds_sig.csv"))
  # disruptive_missense
  disruptive_missense_sig <- results_disruptive_missense_genome[results_disruptive_missense_genome[,"MetaSTAAR-O"]<alpha,,drop=FALSE]
  write.csv(disruptive_missense_sig,file=paste0(output_path,"disruptive_missense_sig.csv"))
  # coding results
  coding_sig <- rbind(plof_sig[,c("Gene name","Chr","Category","#SNV","SKAT-MS(1,25)","Burden-MS(1,1)","ACAT-V-MS(1,25)","MetaSTAAR-O")],
                      missense_sig[,c("Gene name","Chr","Category","#SNV","SKAT-MS(1,25)","Burden-MS(1,1)","ACAT-V-MS(1,25)","MetaSTAAR-O")])
  coding_sig <- rbind(coding_sig,synonymous_sig[,c("Gene name","Chr","Category","#SNV","SKAT-MS(1,25)","Burden-MS(1,1)","ACAT-V-MS(1,25)","MetaSTAAR-O")])
  coding_sig <- rbind(coding_sig,plof_ds_sig[,c("Gene name","Chr","Category","#SNV","SKAT-MS(1,25)","Burden-MS(1,1)","ACAT-V-MS(1,25)","MetaSTAAR-O")])
  coding_sig <- rbind(coding_sig,disruptive_missense_sig[,c("Gene name","Chr","Category","#SNV","SKAT-MS(1,25)","Burden-MS(1,1)","ACAT-V-MS(1,25)","MetaSTAAR-O")])
  write.csv(coding_sig,file=paste0(output_path,"coding_sig.csv"))

  ## manhattan plot
  if(manhattan_plot)
  {
    ### plof
    results_MetaSTAAR <- results_plof_genome[,c(1,2,dim(results_plof_genome)[2])]

    results_m <- c()
    for(i in 1:dim(results_MetaSTAAR)[2])
    {
      results_m <- cbind(results_m,unlist(results_MetaSTAAR[,i]))
    }

    colnames(results_m) <- colnames(results_MetaSTAAR)
    results_m <- data.frame(results_m,stringsAsFactors = FALSE)
    results_m[,2] <- as.numeric(results_m[,2])
    results_m[,3] <- as.numeric(results_m[,3])

    genes_info_manhattan <- dplyr::left_join(genes_info,results_m,by=c("chromosome_name"="Chr","hgnc_symbol"="Gene.name"))

    genes_info_manhattan[is.na(genes_info_manhattan)] <- 1
    colnames(genes_info_manhattan)[dim(genes_info_manhattan)[2]] <- "plof"

    ### plof_ds
    results_MetaSTAAR <- results_plof_ds_genome[,c(1,2,dim(results_plof_ds_genome)[2])]

    results_m <- c()
    for(i in 1:dim(results_MetaSTAAR)[2])
    {
      results_m <- cbind(results_m,unlist(results_MetaSTAAR[,i]))
    }

    colnames(results_m) <- colnames(results_MetaSTAAR)
    results_m <- data.frame(results_m,stringsAsFactors = FALSE)
    results_m[,2] <- as.numeric(results_m[,2])
    results_m[,3] <- as.numeric(results_m[,3])

    genes_info_manhattan <- dplyr::left_join(genes_info_manhattan,results_m,by=c("chromosome_name"="Chr","hgnc_symbol"="Gene.name"))
    genes_info_manhattan[is.na(genes_info_manhattan)] <- 1
    colnames(genes_info_manhattan)[dim(genes_info_manhattan)[2]] <- "plof_ds"

    ### missense
    results_MetaSTAAR <- results_missense_genome[,c(1,2,dim(results_missense_genome)[2]-6)]

    results_m <- c()
    for(i in 1:dim(results_MetaSTAAR)[2])
    {
      results_m <- cbind(results_m,unlist(results_MetaSTAAR[,i]))
    }

    colnames(results_m) <- colnames(results_MetaSTAAR)
    results_m <- data.frame(results_m,stringsAsFactors = FALSE)
    results_m[,2] <- as.numeric(results_m[,2])
    results_m[,3] <- as.numeric(results_m[,3])

    genes_info_manhattan <- dplyr::left_join(genes_info_manhattan,results_m,by=c("chromosome_name"="Chr","hgnc_symbol"="Gene.name"))
    genes_info_manhattan[is.na(genes_info_manhattan)] <- 1
    colnames(genes_info_manhattan)[dim(genes_info_manhattan)[2]] <- "missense"

    ### disruptive_missense
    results_MetaSTAAR <- results_disruptive_missense_genome[,c(1,2,dim(results_disruptive_missense_genome)[2])]

    results_m <- c()
    for(i in 1:dim(results_MetaSTAAR)[2])
    {
      results_m <- cbind(results_m,unlist(results_MetaSTAAR[,i]))
    }

    colnames(results_m) <- colnames(results_MetaSTAAR)
    results_m <- data.frame(results_m,stringsAsFactors = FALSE)
    results_m[,2] <- as.numeric(results_m[,2])
    results_m[,3] <- as.numeric(results_m[,3])

    genes_info_manhattan <- dplyr::left_join(genes_info_manhattan,results_m,by=c("chromosome_name"="Chr","hgnc_symbol"="Gene.name"))
    genes_info_manhattan[is.na(genes_info_manhattan)] <- 1
    colnames(genes_info_manhattan)[dim(genes_info_manhattan)[2]] <- "disruptive_missense"

    ### synonymous
    results_MetaSTAAR <- results_synonymous_genome[,c(1,2,dim(results_synonymous_genome)[2])]

    results_m <- c()
    for(i in 1:dim(results_MetaSTAAR)[2])
    {
      results_m <- cbind(results_m,unlist(results_MetaSTAAR[,i]))
    }

    colnames(results_m) <- colnames(results_MetaSTAAR)
    results_m <- data.frame(results_m,stringsAsFactors = FALSE)
    results_m[,2] <- as.numeric(results_m[,2])
    results_m[,3] <- as.numeric(results_m[,3])

    genes_info_manhattan <- dplyr::left_join(genes_info_manhattan,results_m,by=c("chromosome_name"="Chr","hgnc_symbol"="Gene.name"))
    genes_info_manhattan[is.na(genes_info_manhattan)] <- 1
    colnames(genes_info_manhattan)[dim(genes_info_manhattan)[2]] <- "synonymous"

    ## ylim
    coding_minp <- min(genes_info_manhattan[,(dim(genes_info_manhattan)[2]-4):dim(genes_info_manhattan)[2]])
    min_y <- ceiling(-log10(coding_minp)) + 1

    pch <- c(0,1,2,3,4)
    figure1 <- manhattan_plot(genes_info_manhattan[,2], (genes_info_manhattan[,3]+genes_info_manhattan[,4])/2, genes_info_manhattan$plof,sig.level=alpha,pch=pch[1],col = c("blue4", "orange3"),ylim=c(0,min_y),
                              auto.key=T,key=list(space="top", columns=5, title="Functional Category", cex.title=1, points=TRUE,pch=pch,text=list(c("pLoF","pLoF+D","Missense","Disruptive Missense","Synonymous"))))

    figure2 <- manhattan_plot(genes_info_manhattan[,2], (genes_info_manhattan[,3]+genes_info_manhattan[,4])/2, genes_info_manhattan$plof_ds,sig.level=alpha,pch=pch[2],col = c("blue4", "orange3"),ylim=c(0,min_y),
                              auto.key=T,key=list(space="top", columns=5, title="Functional Category", cex.title=1, points=TRUE,pch=pch,text=list(c("pLoF","pLoF+D","Missense","Disruptive Missense","Synonymous"))))

    figure3 <- manhattan_plot(genes_info_manhattan[,2], (genes_info_manhattan[,3]+genes_info_manhattan[,4])/2, genes_info_manhattan$missense,sig.level=alpha,pch=pch[3],col = c("blue4", "orange3"),ylim=c(0,min_y),
                              auto.key=T,key=list(space="top", columns=5, title="Functional Category", cex.title=1, points=TRUE,pch=pch,text=list(c("pLoF","pLoF+D","Missense","Disruptive Missense","Synonymous"))))

    figure4 <- manhattan_plot(genes_info_manhattan[,2], (genes_info_manhattan[,3]+genes_info_manhattan[,4])/2, genes_info_manhattan$disruptive_missense,sig.level=alpha,pch=pch[4],col = c("blue4", "orange3"),ylim=c(0,min_y),
                              auto.key=T,key=list(space="top", columns=5, title="Functional Category", cex.title=1, points=TRUE,pch=pch,text=list(c("pLoF","pLoF+D","Missense","Disruptive Missense","Synonymous"))))

    figure5 <- manhattan_plot(genes_info_manhattan[,2], (genes_info_manhattan[,3]+genes_info_manhattan[,4])/2, genes_info_manhattan$synonymous,sig.level=alpha,pch=pch[5],col = c("blue4", "orange3"),ylim=c(0,min_y),
                              auto.key=T,key=list(space="top", columns=5, title="Functional Category", cex.title=1, points=TRUE,pch=pch,text=list(c("pLoF","pLoF+D","Missense","Disruptive Missense","Synonymous"))))

    print("Manhattan plot")

    png(paste0(output_path,"gene_centric_coding_manhattan.png"), width = 9, height = 6, units = 'in', res = 600)

    print(figure1)
    print(figure2,newpage = FALSE)
    print(figure3,newpage = FALSE)
    print(figure4,newpage = FALSE)
    print(figure5,newpage = FALSE)

    dev.off()
  }

  ## Q-Q plot
  if(QQ_plot)
  {
    if(!manhattan_plot)
    {
      ### plof
      results_MetaSTAAR <- results_plof_genome[,c(1,2,dim(results_plof_genome)[2])]

      results_m <- c()
      for(i in 1:dim(results_MetaSTAAR)[2])
      {
        results_m <- cbind(results_m,unlist(results_MetaSTAAR[,i]))
      }

      colnames(results_m) <- colnames(results_MetaSTAAR)
      results_m <- data.frame(results_m,stringsAsFactors = FALSE)
      results_m[,2] <- as.numeric(results_m[,2])
      results_m[,3] <- as.numeric(results_m[,3])

      genes_info_manhattan <- dplyr::left_join(genes_info,results_m,by=c("chromosome_name"="Chr","hgnc_symbol"="Gene.name"))

      genes_info_manhattan[is.na(genes_info_manhattan)] <- 1
      colnames(genes_info_manhattan)[dim(genes_info_manhattan)[2]] <- "plof"

      ### plof_ds
      results_MetaSTAAR <- results_plof_ds_genome[,c(1,2,dim(results_plof_ds_genome)[2])]

      results_m <- c()
      for(i in 1:dim(results_MetaSTAAR)[2])
      {
        results_m <- cbind(results_m,unlist(results_MetaSTAAR[,i]))
      }

      colnames(results_m) <- colnames(results_MetaSTAAR)
      results_m <- data.frame(results_m,stringsAsFactors = FALSE)
      results_m[,2] <- as.numeric(results_m[,2])
      results_m[,3] <- as.numeric(results_m[,3])

      genes_info_manhattan <- dplyr::left_join(genes_info_manhattan,results_m,by=c("chromosome_name"="Chr","hgnc_symbol"="Gene.name"))
      genes_info_manhattan[is.na(genes_info_manhattan)] <- 1
      colnames(genes_info_manhattan)[dim(genes_info_manhattan)[2]] <- "plof_ds"

      ### missense
      results_MetaSTAAR <- results_missense_genome[,c(1,2,dim(results_missense_genome)[2]-6)]

      results_m <- c()
      for(i in 1:dim(results_MetaSTAAR)[2])
      {
        results_m <- cbind(results_m,unlist(results_MetaSTAAR[,i]))
      }

      colnames(results_m) <- colnames(results_MetaSTAAR)
      results_m <- data.frame(results_m,stringsAsFactors = FALSE)
      results_m[,2] <- as.numeric(results_m[,2])
      results_m[,3] <- as.numeric(results_m[,3])

      genes_info_manhattan <- dplyr::left_join(genes_info_manhattan,results_m,by=c("chromosome_name"="Chr","hgnc_symbol"="Gene.name"))
      genes_info_manhattan[is.na(genes_info_manhattan)] <- 1
      colnames(genes_info_manhattan)[dim(genes_info_manhattan)[2]] <- "missense"

      ### disruptive_missense
      results_MetaSTAAR <- results_disruptive_missense_genome[,c(1,2,dim(results_disruptive_missense_genome)[2])]

      results_m <- c()
      for(i in 1:dim(results_MetaSTAAR)[2])
      {
        results_m <- cbind(results_m,unlist(results_MetaSTAAR[,i]))
      }

      colnames(results_m) <- colnames(results_MetaSTAAR)
      results_m <- data.frame(results_m,stringsAsFactors = FALSE)
      results_m[,2] <- as.numeric(results_m[,2])
      results_m[,3] <- as.numeric(results_m[,3])

      genes_info_manhattan <- dplyr::left_join(genes_info_manhattan,results_m,by=c("chromosome_name"="Chr","hgnc_symbol"="Gene.name"))
      genes_info_manhattan[is.na(genes_info_manhattan)] <- 1
      colnames(genes_info_manhattan)[dim(genes_info_manhattan)[2]] <- "disruptive_missense"

      ### synonymous
      results_MetaSTAAR <- results_synonymous_genome[,c(1,2,dim(results_synonymous_genome)[2])]

      results_m <- c()
      for(i in 1:dim(results_MetaSTAAR)[2])
      {
        results_m <- cbind(results_m,unlist(results_MetaSTAAR[,i]))
      }

      colnames(results_m) <- colnames(results_MetaSTAAR)
      results_m <- data.frame(results_m,stringsAsFactors = FALSE)
      results_m[,2] <- as.numeric(results_m[,2])
      results_m[,3] <- as.numeric(results_m[,3])

      genes_info_manhattan <- dplyr::left_join(genes_info_manhattan,results_m,by=c("chromosome_name"="Chr","hgnc_symbol"="Gene.name"))
      genes_info_manhattan[is.na(genes_info_manhattan)] <- 1
      colnames(genes_info_manhattan)[dim(genes_info_manhattan)[2]] <- "synonymous"

      ## ylim
      coding_minp <- min(genes_info_manhattan[,(dim(genes_info_manhattan)[2]-4):dim(genes_info_manhattan)[2]])
      min_y <- ceiling(-log10(coding_minp)) + 1
    }

    print("Q-Q plot")
    cex_point <- 1

    png(paste0(output_path,"gene_centric_coding_qqplot.png"), width = 9, height = 9, units = 'in', res = 600)

    ### plof
    ## remove unconverged p-values
    observed <- sort(genes_info_manhattan$plof[genes_info_manhattan$plof < 1])
    lobs <- -(log10(observed))

    expected <- c(1:length(observed))
    lexp <- -(log10(expected / (length(expected)+1)))

    # par(mar=c(5,6,4,4))
    plot(lexp,lobs,pch=0, cex=cex_point, xlim = c(0, 5), ylim = c(0, min_y),
         xlab = expression(Expected ~ ~-log[10](italic(p))), ylab = expression(Observed ~ ~-log[10](italic(p))),
         font.lab=2,cex.lab=1,cex.axis=1,font.axis=2)

    abline(0, 1, col="red",lwd=1)

    ### plof_ds
    ## remove unconverged p-values
    observed <- sort(genes_info_manhattan$plof_ds[genes_info_manhattan$plof_ds < 1])

    lobs <- -(log10(observed))

    expected <- c(1:length(observed))
    lexp <- -(log10(expected / (length(expected)+1)))

    par(new=T)
    # par(mar=c(5,6,4,4))
    plot(lexp,lobs,pch=1, cex=cex_point, xlim = c(0, 5), ylim = c(0, min_y),
         xlab = expression(Expected ~ ~-log[10](italic(p))), ylab = expression(Observed ~ ~-log[10](italic(p))),
         font.lab=2,cex.lab=1,cex.axis=1,font.axis=2)

    abline(0, 1, col="red",lwd=1)

    ### missense
    ## remove unconverged p-values
    observed <- sort(genes_info_manhattan$missense[genes_info_manhattan$missense < 1])

    lobs <- -(log10(observed))

    expected <- c(1:length(observed))
    lexp <- -(log10(expected / (length(expected)+1)))

    par(new=T)
    # par(mar=c(5,6,4,4))
    plot(lexp,lobs,pch=2, cex=cex_point, xlim = c(0, 5), ylim = c(0, min_y),
         xlab = expression(Expected ~ ~-log[10](italic(p))), ylab = expression(Observed ~ ~-log[10](italic(p))),
         font.lab=2,cex.lab=1,cex.axis=1,font.axis=2)

    abline(0, 1, col="red",lwd=1)

    ### disruptive_missense
    ## remove unconverged p-values
    observed <- sort(genes_info_manhattan$disruptive_missense[genes_info_manhattan$disruptive_missense < 1])


    lobs <- -(log10(observed))

    expected <- c(1:length(observed))
    lexp <- -(log10(expected / (length(expected)+1)))

    par(new=T)
    # par(mar=c(5,6,4,4))
    plot(lexp,lobs,pch=3, cex=cex_point, xlim = c(0, 5), ylim = c(0, min_y),
         xlab = expression(Expected ~ ~-log[10](italic(p))), ylab = expression(Observed ~ ~-log[10](italic(p))),
         font.lab=2,cex.lab=1,cex.axis=1,font.axis=2)

    abline(0, 1, col="red",lwd=1)

    ### synonymous
    ## remove unconverged p-values
    observed <- sort(genes_info_manhattan$synonymous[genes_info_manhattan$synonymous < 1])

    lobs <- -(log10(observed))

    expected <- c(1:length(observed))
    lexp <- -(log10(expected / (length(expected)+1)))

    par(new=T)
    # par(mar=c(5,6,4,4))
    plot(lexp,lobs,pch=4, cex=cex_point, xlim = c(0, 5), ylim = c(0, min_y),
         xlab = expression(Expected ~ ~-log[10](italic(p))), ylab = expression(Observed ~ ~-log[10](italic(p))),
         font.lab=2,cex.lab=1,cex.axis=1,font.axis=2)

    abline(0, 1, col="red",lwd=1)

    legend("topleft",legend=c("pLoF","pLoF+D","Missense","Disruptive Missense","Synonymous"),ncol=1,bty="o",box.lwd=1,pch=0:4,cex=1,text.font=2)

    dev.off()
  }

}

###########################################################
#           User Input
###########################################################
## results path
input_path <- Meta_SumStats
output_path <- paste0(OUTPUT_PATH,"/")
## number of jobs
gene_centric_coding_jobs_num <- 381
## results name
gene_centric_results_name <- "/UKB_AoU_Coding"

## alpha level
alpha <- 5E-07

###########################################################
#           Main Function
###########################################################
Gene_Centric_Coding_Results_Summary_meta(gene_centric_coding_jobs_num=gene_centric_coding_jobs_num,
                                         input_path=input_path,output_path=output_path,
                                         gene_centric_results_name=gene_centric_results_name,
                                         alpha=alpha,manhattan_plot=TRUE,QQ_plot=TRUE)

Overwriting Meta_Coding_UKB_AoU_Summary.R


In [27]:
%%writefile Meta_Coding_UKB_AoU_Summary.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${Meta_SumStats} ${OUTPUT_PATH}

Overwriting Meta_Coding_UKB_AoU_Summary.sh


In [28]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)


tasks <- rbind(tasks, data.frame(
            '--input-recursive Meta_SumStats'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/UKB_AoU_Meta",
            '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/Meta_Coding_UKB_AoU_Summary.R",
            '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/Meta_Results",
            check.names = FALSE
        )) 
    
  
write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [29]:
!Rscript score_task.R

In [30]:
!gsutil -m cp Meta_Coding_UKB_AoU_Summary.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/MetaSTAAR/

Copying file://Meta_Coding_UKB_AoU_Summary.R [Content-Type=application/octet-stream]...
/ [1/1 files][ 18.6 KiB/ 18.6 KiB] 100% Done                                    
Operation completed over 1 objects/18.6 KiB.                                     


In [31]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 100 \
  --boot-disk-size 25 \
  --min-ram 16 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script Meta_Coding_UKB_AoU_Summary.sh \
  --tasks score_task.txt

Job properties:
  job-id: meta-codin--williamsjacr--241120-152842-90
  job-name: meta-coding-ukb-aou-summary
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/3908227709257602441
Launched job-id: meta-codin--williamsjacr--241120-152842-90
1 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'meta-codin--williamsjacr--241120-152842-90' --users 'williamsjacr' --status '*'
To cancel the job, run:
  ddel --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'meta-codin--williamsjacr--241120-152842-90' --users 'williamsjacr'


In [35]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'meta-codin--williamsjacr--241120-152842-90' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
meta-coding-...       1  Success   11-20 16:28:29

